# import

In [1]:
import os
import gc
import time
import copy
import random
import math
import glob
import cv2
import warnings
from pathlib import Path
from collections import defaultdict

import numpy as np
import pandas as pd
import polars as pl
import pyarrow
import joblib
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp


import timm

import albumentations as A
from albumentations.pytorch import ToTensorV2

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import (
    GroupKFold,
    StratifiedGroupKFold,
    KFold,
    StratifiedKFold
)
from sklearn.preprocessing import (
    OneHotEncoder,
    LabelEncoder
)
from sklearn.metrics import (
    roc_auc_score,
    mean_squared_error,
    roc_curve,
    auc,
    confusion_matrix
)
from sklearn.feature_selection import (
    SelectKBest,
    chi2,
    mutual_info_classif,
    VarianceThreshold
)
from sklearn.base import BaseEstimator, TransformerMixin

from imblearn.under_sampling import (
    RandomUnderSampler,
    NearMiss,
    ClusterCentroids,
    TomekLinks,
    EditedNearestNeighbours
)
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import lightgbm as lgb
import catboost as cb
import xgboost as xgb

import optuna
from optuna.samplers import TPESampler

import PIL

import os
import gc
import time
import copy
import optuna

from pathlib import Path

import numpy as np, pandas as pd, polars as pl

from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import VotingClassifier

from sklearn.utils import resample

from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler

import lightgbm as lgb, catboost as cb, xgboost as xgb

from optuna.samplers import TPESampler

from imblearn.under_sampling import NearMiss
from imblearn.under_sampling import ClusterCentroids
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import EditedNearestNeighbours
# from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.under_sampling import NearMiss, TomekLinks
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif, VarianceThreshold
from sklearn.base import BaseEstimator, TransformerMixin

import warnings
from sklearn.exceptions import FitFailedWarning

# 忽略所有的警告
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FitFailedWarning)
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

import ast

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import io
import os
import h5py
import pandas as pd
from torchvision import transforms
import numpy as np
import timm
import tqdm
from PIL import Image
from io import BytesIO

import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/albumentations/check_version.py:49: UserWarning: Error fetching version info <urlopen error [Errno -3] Temporary failure in name resolution>
  data = fetch_version_info()


# Image Features Prediction

In [2]:
CONFIG = {
    "img_size": 224,
    "model_name1": "swin_base_patch4_window7_224",
    "model_name2": "convnext_base_in22k",
    "model_name3": "swin_base_patch4_window12_384",
    #"model_load_path": '',
    "model_load_path1": '/kaggle/input/miccai_model/pytorch/default/1/1_Positive_swin_base_patch4_window7_224_0.153.pt',
    "model_load_path2": '/kaggle/input/miccai_model/pytorch/default/1/2_swin_base_patch4_window7_224_0.155.pt',
    "model_load_path3": '/kaggle/input/miccai_model/pytorch/default/1/3_Positive_Convnext_base_224_0.157.pt',
    "model_load_path4": '/kaggle/input/miccai_model/pytorch/default/1/4_convnext_base_in22k_224_0.156.pt',
    "model_load_path5": '/kaggle/input/miccai_model/pytorch/default/1/5_Swin_efficientnet_B0_positive_distribition_change_0.147.pt',
    "model_load_path6": '/kaggle/input/miccai_model/pytorch/default/1/6_swin_base_patch4_window12_384_0.158.pt',
    "model_load_path7": '/kaggle/input/miccai_model/pytorch/default/1/7_Fusion_swin_base_patch4_window7_224_0.153.pt',
    "model_load_path8": '/kaggle/input/miccai_model/pytorch/default/1/8_Fusion_convnext_base_224_0.147.pt',
    "valid_batch_size": 16,
    "inference_batch_size": 16,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    "img_resize": False,
    "test_metadata_path": '/kaggle/input/isic-2024-challenge/test-metadata.csv', #'/kaggle/input/isic-2024-challenge/train-metadata.csv',
    "test_images_h5_path": '/kaggle/input/isic-2024-challenge/test-image.hdf5', #'/kaggle/input/isic-2024-challenge/train-image.hdf5',
    "train_metadata_path": '/kaggle/input/isic-2024-challenge/train-metadata.csv',
    "train_images_h5_path": '/kaggle/input/isic-2024-challenge/train-image.hdf5',
    "submission_path": '/kaggle/input/isic-2024-challenge/sample_submission.csv',
}

In [3]:
data_preprocess = A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
        max_pixel_value=255.0,
        p=1.0
    ),
        ToTensorV2()], p=1.)

def read_images_from_hdf5(file_path):
    with h5py.File(file_path, 'r') as file:
        ids_list = list(file.keys())        
        ids_images = {}
        for img_id in tqdm.tqdm(ids_list):
            image_data = file[img_id][()]
            image = Image.open(io.BytesIO(image_data))
            ids_images[img_id] = np.array(image)
    return ids_images

In [4]:
# Model Zoo

class ISICModel(nn.Module):
    def __init__(self, model_name, num_classes=1, pretrained=False):
        super(ISICModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        in_features = self.model.head.fc.in_features
        self.sigmoid = nn.Sigmoid()
        self.model.head.fc = nn.Linear(in_features, num_classes)

    def forward(self, images):
        output = self.model(images)
        return output
###Swin_positive_distribition_change_0.153
model1 = ISICModel(CONFIG['model_name1'])
model1.to(CONFIG['device']);
model1.load_state_dict(torch.load(CONFIG['model_load_path1'], map_location=torch.device(CONFIG['device'])))
###Swin_past_data_argumentation_0.155
model2 = ISICModel(CONFIG['model_name1'])
model2.to(CONFIG['device']);
model2.load_state_dict(torch.load(CONFIG['model_load_path2'], map_location=torch.device(CONFIG['device'])))
###Convnext_positive_distribition_change_0.157
model3 = ISICModel(CONFIG['model_name2'])
model3.to(CONFIG['device']);
model3.load_state_dict(torch.load(CONFIG['model_load_path3'], map_location=torch.device(CONFIG['device'])))
###Convnext_past_data_argumentation_0.156
model4 = ISICModel(CONFIG['model_name2'])
model4.to(CONFIG['device']);
model4.load_state_dict(torch.load(CONFIG['model_load_path4'], map_location=torch.device(CONFIG['device'])))
### Swin_Best_0.158
### 不一定要有这个模型
model6 = ISICModel(CONFIG['model_name3'])
model6.to(CONFIG['device']);
model6.load_state_dict(torch.load(CONFIG['model_load_path6'], map_location=torch.device(CONFIG['device'])))

class ISICModel_CNN_Transformer(nn.Module):
    def __init__(self, model_name, num_classes=1, pretrained=False):
        super(ISICModel_CNN_Transformer, self).__init__()
        self.Relu=nn.ReLU()
        self.dropout=nn.Dropout(0.1)
        self.model = timm.create_model("swin_base_patch4_window7_224", pretrained=pretrained )
        self.model2 = timm.create_model("efficientnet_b0", pretrained=pretrained )
        in_features = self.model2.classifier.in_features+self.model.head.fc.in_features
        self.model.head.fc = nn.Identity()
        self.model2.classifier = nn.Identity()
        self.sigmoid = nn.Sigmoid()
        self.nn=nn.Linear(in_features,1024)
        self.nn2 = nn.Linear(1024, num_classes)

    def forward(self, images):
        output = self.model(images)
        output1=self.model2(images)
        output=torch.cat((output1, output), dim=1)
        output=self.Relu(output)
        output=self.dropout(output)
        output=self.nn(output)
        output=self.Relu(output)
        output=self.dropout(output)
        output=self.nn2(output)
        return output
### Swin_efficientnet_B0_positive_distribition_change_0.147
model5 = ISICModel_CNN_Transformer(CONFIG['model_name1'])
model5.to(CONFIG['device']);
model5.load_state_dict(torch.load(CONFIG['model_load_path5'], map_location=torch.device(CONFIG['device'])))

class ISICModel_Fusion(nn.Module):
    def __init__(self, model_name, num_classes=1, pretrained=False, checkpoint_path=None):
        super(ISICModel_Fusion, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        in_features = self.model.head.fc.in_features
        self.Relu=nn.ReLU()
        self.dropout=nn.Dropout(0.1)
        self.linear = nn.Linear(63, 256)
        self.model.head.fc = nn.Identity()
        self.sigmoid = nn.Sigmoid()
        self.linear1 = nn.Linear(in_features+256, 1000)
        self.linear2=nn.Linear(1000,1)

    def forward(self, images,tables):
        x=self.linear(tables)
        output = self.model(images)
        output=torch.cat((output,x),dim=1)
        output=self.linear1(output)
        output=self.Relu(output)
        output=self.dropout(output)
        output=self.linear2(output)
        return output
### Swin_Modal_Fusion_0.153
model7 = ISICModel_Fusion(CONFIG['model_name1'])
model7.to(CONFIG['device']);
model7.load_state_dict(torch.load(CONFIG['model_load_path7'], map_location=torch.device(CONFIG['device'])))
### Convnext_Modal_Fusion_0.147
model8 = ISICModel_Fusion(CONFIG['model_name2'])
model8.to(CONFIG['device']);
model8.load_state_dict(torch.load(CONFIG['model_load_path8'], map_location=torch.device(CONFIG['device'])))

<All keys matched successfully>

In [5]:
# Inference Function and Class

class ISIC2024Dataset(Dataset):
    def __init__(self, 
                 metadata: pd.DataFrame,
                 ids_images_pth, #: dict,
                 transforms=None):
        
        self.metadata = metadata
        self.fp_hdf = h5py.File(ids_images_pth, mode="r")
        self.transforms = transforms

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, item):
        isic_row = self.metadata.iloc[item]
        isic_id = isic_row.isic_id
        image = np.array( Image.open(BytesIO(self.fp_hdf[isic_id][()])) )
        if self.transforms:
            image = self.transforms(image=image)["image"]
        return image

def inference(valid_loader, model, device):
    model.eval() 
    preds = []
    with tqdm.notebook.tqdm(valid_loader, unit="inference_batch", desc='Inference') as tqdm_valid_loader:
        for step, batch in enumerate(tqdm_valid_loader):
            X = batch.to(device).float()
            with torch.no_grad():
                batch_size = X.size(0)
                y_preds = model(X).squeeze()
                preds.append(y_preds.sigmoid().cpu().numpy())
                
    preds = np.concatenate(preds, 0)
    return preds

class ISIC2024Dataset_for_fusion(Dataset):
    def __init__(self, 
                 metadata: pd.DataFrame,
                 ids_images_pth, #: dict,
                 transforms=None):
        
        self.metadata = metadata
        self.fp_hdf = h5py.File(ids_images_pth, mode="r")
        self.transforms = transforms

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, item):
        isic_row = self.metadata.iloc[item]
        isic_id = isic_row.isic_id
        image = np.array( Image.open(BytesIO(self.fp_hdf[isic_id][()])) )
        if self.transforms:
            image = self.transforms(image=image)["image"]
        row_data = self.metadata.iloc[item].drop(['target', 'isic_id'])


        # 假设 row_data 已经是全部数值型数据
        info_tensor = row_data.values.astype(np.float32)
        return {
            'image': image,
            "table":info_tensor}

def inference_for_fusion(valid_loader, model, device):
    model.eval() 
    preds = []
    with tqdm.notebook.tqdm(valid_loader, unit="inference_batch", desc='Inference') as tqdm_valid_loader:
        for step, batch in enumerate(tqdm_valid_loader):
            X = batch['image'].to(device, dtype=torch.float)
            Y = batch['table'].to(device, dtype=torch.float)
            with torch.no_grad():
                y_preds = model(X,Y).squeeze()
                preds.append(y_preds.sigmoid().cpu().numpy())
                
    preds = np.concatenate(preds, 0)
    return preds

In [6]:
import pandas as pd
import numpy as np
df_train = pd.read_csv("/kaggle/input/isic-2024-challenge/train-metadata.csv")
df_test = pd.read_csv('/kaggle/input/isic-2024-challenge/test-metadata.csv')
df_test['target'] = 0 # dummy
feature_name=['isic_id',
        'age_approx',
        'sex',
        'anatom_site_general', 'clin_size_long_diam_mm',
        'tbp_tile_type',
        'tbp_lv_A', 'tbp_lv_Aext',
        'tbp_lv_B', 'tbp_lv_Bext',
        'tbp_lv_C', 'tbp_lv_Cext',
        'tbp_lv_H', 'tbp_lv_Hext',
        'tbp_lv_L', 'tbp_lv_Lext',
        'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio',
        'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB',
        'tbp_lv_deltaL', 'tbp_lv_deltaLB',
        'tbp_lv_deltaLBnorm', 'tbp_lv_eccentricity',
        'tbp_lv_location', 'tbp_lv_location_simple',
        'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence',
        'tbp_lv_norm_border', 'tbp_lv_norm_color',
        'tbp_lv_perimeterMM',
        'tbp_lv_radial_color_std_max',
        'tbp_lv_stdL', 'tbp_lv_stdLExt',
        'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle',
        'tbp_lv_x', 'tbp_lv_y', 'tbp_lv_z',
        'target'
        ]
df_train=df_train[feature_name]
df_test=df_test[feature_name]
def data_cleaning(df):
    df['age_approx'] = df['age_approx'].fillna(df['age_approx'].median())
    df.drop(columns=['tbp_lv_location_simple'], inplace=True)
    df['tbp_lv_location'] = df['tbp_lv_location'].replace('Unknown', np.nan)
    return df

# 清洗数据
df_train_clean = data_cleaning(df_train)
df_test_clean = data_cleaning(df_test)

# 合并训练集和测试集进行编码
combined_df = pd.concat([df_train_clean, df_test_clean], ignore_index=True)

# 获取分类列的列表
categorical_cols = ['anatom_site_general', 'tbp_lv_location', 'tbp_tile_type', 'sex']

# 进行独热编码
combined_df = pd.get_dummies(combined_df, columns=categorical_cols)

# 拆分回训练集和测试集
df_train_encoded = combined_df.iloc[:len(df_train_clean)]
df_test_encoded = combined_df.iloc[len(df_train_clean):]

# 还原索引
df_train_encoded.index = df_train.index
df_test_encoded.index = df_test.index

# 检查处理结果
df_train=df_train_encoded
df_test=df_test_encoded
df_train.to_csv("train.csv")
df_test.to_csv("test.csv")
import pandas as pd
from sklearn.preprocessing import StandardScaler


# 1.转换数据类型
# 排除 'isic_id' 和 'target' 两个列，选择其余的列
cols_to_convert = [col for col in df_train.columns if col not in ('isic_id', 'target')]

df_train[cols_to_convert] = df_train[cols_to_convert].astype('float64')
df_test[cols_to_convert] = df_test[cols_to_convert].astype('float64')

# 2. 对 df_train 进行标准化
scaler = StandardScaler()
df_train_scaled = df_train.copy()
df_test_scaled = df_test.copy()

# 标准化训练数据
df_train_scaled[cols_to_convert] = scaler.fit_transform(df_train_scaled[cols_to_convert])

# 标准化测试数据
df_test_scaled[cols_to_convert] = scaler.transform(df_test_scaled[cols_to_convert])

df_train_scaled.to_csv("train_scaled.csv", index=False)
df_test_scaled.to_csv("test_scaled.csv", index=False)

In [7]:
# Inference
metadata_test = pd.read_csv(CONFIG['test_metadata_path'])
TEST_HDF = "/kaggle/input/isic-2024-challenge/test-image.hdf5"
dataset_test = ISIC2024Dataset(metadata_test, TEST_HDF,data_preprocess)
test_dataloader = torch.utils.data.DataLoader(
        dataset_test,
        batch_size = CONFIG['inference_batch_size'],
        num_workers = 16,
        shuffle = False,
        pin_memory = True,
    )
preds = inference(test_dataloader, model1, CONFIG['device'])
print(preds)
submission = pd.read_csv(CONFIG['submission_path'])
submission['isic_id'] = metadata_test['isic_id'].to_list()
submission['target_Swin_positive_distribition_change_0.153'] = preds
submission.to_csv('submission1_Swin_positive_distribition_change_0.153.csv', index=False)

Inference:   0%|          | 0/1 [00:00<?, ?inference_batch/s]

[0.18987924 0.12141959 0.09707573]


In [8]:
metadata_test = pd.read_csv(CONFIG['test_metadata_path'])
TEST_HDF = "/kaggle/input/isic-2024-challenge/test-image.hdf5"
dataset_test = ISIC2024Dataset(metadata_test, TEST_HDF,data_preprocess)
test_dataloader = torch.utils.data.DataLoader(
        dataset_test,
        batch_size=CONFIG['inference_batch_size'],
        num_workers=16,
        shuffle=False,
        pin_memory=True,
    )
preds = inference(test_dataloader, model2, CONFIG['device'])
print(preds)
submission = pd.read_csv(CONFIG['submission_path'])
submission['isic_id'] = metadata_test['isic_id'].to_list()
submission['target_Swin_past_data_argumentation_0.155'] = preds
submission.to_csv('submission2_Swin_past_data_argumentation_0.155.csv', index=False)

Inference:   0%|          | 0/1 [00:00<?, ?inference_batch/s]

[0.27631372 0.08312987 0.1352154 ]


In [9]:
metadata_test = pd.read_csv(CONFIG['test_metadata_path'])
TEST_HDF = "/kaggle/input/isic-2024-challenge/test-image.hdf5"
dataset_test = ISIC2024Dataset(metadata_test, TEST_HDF,data_preprocess)
test_dataloader = torch.utils.data.DataLoader(
        dataset_test,
        batch_size=CONFIG['inference_batch_size'],
        num_workers=16,
        shuffle=False,
        pin_memory=True,
    )
preds = inference(test_dataloader, model3, CONFIG['device'])
print(preds)
submission = pd.read_csv(CONFIG['submission_path'])
submission['isic_id'] = metadata_test['isic_id'].to_list()
submission['target_Convnext_positive_distribition_change_0.157'] = preds
submission.to_csv('submission3_Convnext_positive_distribition_change_0.157.csv', index=False)

Inference:   0%|          | 0/1 [00:00<?, ?inference_batch/s]

[0.22985058 0.10410498 0.17126496]


In [10]:
metadata_test = pd.read_csv(CONFIG['test_metadata_path'])
TEST_HDF = "/kaggle/input/isic-2024-challenge/test-image.hdf5"
dataset_test = ISIC2024Dataset(metadata_test, TEST_HDF,data_preprocess)
test_dataloader = torch.utils.data.DataLoader(
        dataset_test,
        batch_size=CONFIG['inference_batch_size'],
        num_workers=16,
        shuffle=False,
        pin_memory=True,
    )
preds = inference(test_dataloader, model4, CONFIG['device'])
print(preds)
submission = pd.read_csv(CONFIG['submission_path'])
submission['isic_id'] = metadata_test['isic_id'].to_list()
submission['target_Convnext_past_data_argumentation_0.156'] = preds
submission.to_csv('submission4_Convnext_past_data_argumentation_0.156.csv', index=False)

Inference:   0%|          | 0/1 [00:00<?, ?inference_batch/s]

[0.19208409 0.10344937 0.15131679]


In [11]:
metadata_test = pd.read_csv(CONFIG['test_metadata_path'])
TEST_HDF = "/kaggle/input/isic-2024-challenge/test-image.hdf5"
dataset_test = ISIC2024Dataset(metadata_test, TEST_HDF,data_preprocess)
test_dataloader = torch.utils.data.DataLoader(
        dataset_test,
        batch_size=CONFIG['inference_batch_size'],
        num_workers=16,
        shuffle=False,
        pin_memory=True,
    )
preds = inference(test_dataloader, model5, CONFIG['device'])
print(preds)
submission = pd.read_csv(CONFIG['submission_path'])
submission['isic_id'] = metadata_test['isic_id'].to_list()
submission['target_Swin_efficientnet_B0_positive_distribition_change_0.147'] = preds
submission.to_csv('submission5_Swin_efficientnet_B0_positive_distribition_change_0.147.csv', index=False)

Inference:   0%|          | 0/1 [00:00<?, ?inference_batch/s]

[0.09087959 0.07155439 0.08443996]


In [12]:
metadata_test = pd.read_csv("test_scaled.csv")
TEST_HDF = "/kaggle/input/isic-2024-challenge/test-image.hdf5"
dataset_test = ISIC2024Dataset_for_fusion(metadata_test, TEST_HDF,data_preprocess)
test_dataloader = torch.utils.data.DataLoader(
        dataset_test,
        batch_size=CONFIG['inference_batch_size'],
        num_workers=16,
        shuffle=False,
        pin_memory=True,
    )
preds = inference_for_fusion(test_dataloader, model7, CONFIG['device'])
print(preds)
submission = pd.read_csv(CONFIG['submission_path'])
submission['isic_id'] = metadata_test['isic_id'].to_list()
submission['target_Swin_Modal_Fusion_0.153'] = preds
submission.to_csv('submission7_Swin_Modal_Fusion_0.153.csv', index=False)

Inference:   0%|          | 0/1 [00:00<?, ?inference_batch/s]

[0.03302179 0.02131753 0.07090748]


In [13]:
metadata_test = pd.read_csv("test_scaled.csv")
TEST_HDF = "/kaggle/input/isic-2024-challenge/test-image.hdf5"
dataset_test = ISIC2024Dataset_for_fusion(metadata_test, TEST_HDF,data_preprocess)
test_dataloader = torch.utils.data.DataLoader(
        dataset_test,
        batch_size=CONFIG['inference_batch_size'],
        num_workers=16,
        shuffle=False,
        pin_memory=True,
    )
preds = inference_for_fusion(test_dataloader, model8, CONFIG['device'])
print(preds)
submission = pd.read_csv(CONFIG['submission_path'])
submission['isic_id'] = metadata_test['isic_id'].to_list()
submission['target_Convnext_Modal_Fusion_0.147'] = preds
submission.to_csv('submission8_Convnext_Modal_Fusion_0.147.csv', index=False)

Inference:   0%|          | 0/1 [00:00<?, ?inference_batch/s]

[0.03376736 0.02192784 0.05040231]


In [14]:
data_preprocess = A.Compose([
    A.Resize(384, 384),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
        max_pixel_value=255.0,
        p=1.0
    ),
        ToTensorV2()], p=1.)

metadata_test = pd.read_csv(CONFIG['test_metadata_path'])
TEST_HDF = "/kaggle/input/isic-2024-challenge/test-image.hdf5"
dataset_test = ISIC2024Dataset(metadata_test, TEST_HDF,data_preprocess)
test_dataloader = torch.utils.data.DataLoader(
        dataset_test,
        batch_size=CONFIG['inference_batch_size'],
        num_workers=16,
        shuffle=False,
        pin_memory=True,
    )
preds = inference(test_dataloader, model6, CONFIG['device'])
print(preds)
submission = pd.read_csv(CONFIG['submission_path'])
submission['isic_id'] = metadata_test['isic_id'].to_list()
submission['target_Swin_Best_0.158'] = preds
submission.to_csv('submission6_Swin_Best_0.158.csv', index=False)

Inference:   0%|          | 0/1 [00:00<?, ?inference_batch/s]

[0.17314175 0.1135863  0.14173919]


# Data

In [15]:
root = Path('/kaggle/input/isic-2024-challenge')

train_path = root / 'train-metadata.csv'
test_path = root / 'test-metadata.csv'
subm_path = root / 'sample_submission.csv'

id_col = 'isic_id'
target_col = 'target'
group_col = 'patient_id'

err = 1e-5
seed = 42

In [16]:
num_cols = [
    'age_approx',                        # Approximate age of patient at time of imaging.
    'clin_size_long_diam_mm',            # Maximum diameter of the lesion (mm).+
    'tbp_lv_A',                          # A inside  lesion.+
    'tbp_lv_Aext',                       # A outside lesion.+
    'tbp_lv_B',                          # B inside  lesion.+
    'tbp_lv_Bext',                       # B outside lesion.+ 
    'tbp_lv_C',                          # Chroma inside  lesion.+
    'tbp_lv_Cext',                       # Chroma outside lesion.+
    'tbp_lv_H',                          # Hue inside the lesion; calculated as the angle of A* and B* in LAB* color space. Typical values range from 25 (red) to 75 (brown).+
    'tbp_lv_Hext',                       # Hue outside lesion.+
    'tbp_lv_L',                          # L inside lesion.+
    'tbp_lv_Lext',                       # L outside lesion.+
    'tbp_lv_areaMM2',                    # Area of lesion (mm^2).+
    'tbp_lv_area_perim_ratio',           # Border jaggedness, the ratio between lesions perimeter and area. Circular lesions will have low values; irregular shaped lesions will have higher values. Values range 0-10.+
    'tbp_lv_color_std_mean',             # Color irregularity, calculated as the variance of colors within the lesion's boundary.
    'tbp_lv_deltaA',                     # Average A contrast (inside vs. outside lesion).+
    'tbp_lv_deltaB',                     # Average B contrast (inside vs. outside lesion).+
    'tbp_lv_deltaL',                     # Average L contrast (inside vs. outside lesion).+
    'tbp_lv_deltaLB',                    #
    'tbp_lv_deltaLBnorm',                # Contrast between the lesion and its immediate surrounding skin. Low contrast lesions tend to be faintly visible such as freckles; high contrast lesions tend to be those with darker pigment. Calculated as the average delta LB of the lesion relative to its immediate background in LAB* color space. Typical values range from 5.5 to 25.+
    'tbp_lv_eccentricity',               # Eccentricity.+
    'tbp_lv_minorAxisMM',                # Smallest lesion diameter (mm).+
    'tbp_lv_nevi_confidence',            # Nevus confidence score (0-100 scale) is a convolutional neural network classifier estimated probability that the lesion is a nevus. The neural network was trained on approximately 57,000 lesions that were classified and labeled by a dermatologist.+,++
    'tbp_lv_norm_border',                # Border irregularity (0-10 scale); the normalized average of border jaggedness and asymmetry.+
    'tbp_lv_norm_color',                 # Color variation (0-10 scale); the normalized average of color asymmetry and color irregularity.+
    'tbp_lv_perimeterMM',                # Perimeter of lesion (mm).+
    'tbp_lv_radial_color_std_max',       # Color asymmetry, a measure of asymmetry of the spatial distribution of color within the lesion. This score is calculated by looking at the average standard deviation in LAB* color space within concentric rings originating from the lesion center. Values range 0-10.+
    'tbp_lv_stdL',                       # Standard deviation of L inside  lesion.+
    'tbp_lv_stdLExt',                    # Standard deviation of L outside lesion.+
    'tbp_lv_symm_2axis',                 # Border asymmetry; a measure of asymmetry of the lesion's contour about an axis perpendicular to the lesion's most symmetric axis. Lesions with two axes of symmetry will therefore have low scores (more symmetric), while lesions with only one or zero axes of symmetry will have higher scores (less symmetric). This score is calculated by comparing opposite halves of the lesion contour over many degrees of rotation. The angle where the halves are most similar identifies the principal axis of symmetry, while the second axis of symmetry is perpendicular to the principal axis. Border asymmetry is reported as the asymmetry value about this second axis. Values range 0-10.+
    'tbp_lv_symm_2axis_angle',           # Lesion border asymmetry angle.+
    'tbp_lv_x',                          # X-coordinate of the lesion on 3D TBP.+
    'tbp_lv_y',                          # Y-coordinate of the lesion on 3D TBP.+
    'tbp_lv_z',                          # Z-coordinate of the lesion on 3D TBP.+
]

new_num_cols = [
    'lesion_size_ratio',                 # tbp_lv_minorAxisMM      / clin_size_long_diam_mm
    'lesion_shape_index',                # tbp_lv_areaMM2          / tbp_lv_perimeterMM **2
    'hue_contrast',                      # tbp_lv_H                - tbp_lv_Hext              abs
    'luminance_contrast',                # tbp_lv_L                - tbp_lv_Lext              abs
    'lesion_color_difference',           # tbp_lv_deltaA **2       + tbp_lv_deltaB **2 + tbp_lv_deltaL **2  sqrt  
    'border_complexity',                 # tbp_lv_norm_border      + tbp_lv_symm_2axis
    'color_uniformity',                  # tbp_lv_color_std_mean   / tbp_lv_radial_color_std_max

    'position_distance_3d',              # tbp_lv_x **2 + tbp_lv_y **2 + tbp_lv_z **2  sqrt
    'perimeter_to_area_ratio',           # tbp_lv_perimeterMM      / tbp_lv_areaMM2
    'area_to_perimeter_ratio',           # tbp_lv_areaMM2          / tbp_lv_perimeterMM
    'lesion_visibility_score',           # tbp_lv_deltaLBnorm      + tbp_lv_norm_color
    'symmetry_border_consistency',       # tbp_lv_symm_2axis       * tbp_lv_norm_border
    'consistency_symmetry_border',       # tbp_lv_symm_2axis       * tbp_lv_norm_border / (tbp_lv_symm_2axis + tbp_lv_norm_border)

    'color_consistency',                 # tbp_lv_stdL             / tbp_lv_Lext
    'consistency_color',                 # tbp_lv_stdL*tbp_lv_Lext / tbp_lv_stdL + tbp_lv_Lext
    'size_age_interaction',              # clin_size_long_diam_mm  * age_approx
    'hue_color_std_interaction',         # tbp_lv_H                * tbp_lv_color_std_mean
    'lesion_severity_index',             # tbp_lv_norm_border      + tbp_lv_norm_color + tbp_lv_eccentricity / 3
    'shape_complexity_index',            # border_complexity       + lesion_shape_index
    'color_contrast_index',              # tbp_lv_deltaA + tbp_lv_deltaB + tbp_lv_deltaL + tbp_lv_deltaLBnorm

    'log_lesion_area',                   # tbp_lv_areaMM2          + 1  np.log
    'normalized_lesion_size',            # clin_size_long_diam_mm  / age_approx
    'mean_hue_difference',               # tbp_lv_H                + tbp_lv_Hext    / 2
    'std_dev_contrast',                  # tbp_lv_deltaA **2 + tbp_lv_deltaB **2 + tbp_lv_deltaL **2   / 3  np.sqrt
    'color_shape_composite_index',       # tbp_lv_color_std_mean   + bp_lv_area_perim_ratio + tbp_lv_symm_2axis   / 3
    'lesion_orientation_3d',             # tbp_lv_y                , tbp_lv_x  np.arctan2
    'overall_color_difference',          # tbp_lv_deltaA           + tbp_lv_deltaB + tbp_lv_deltaL   / 3

    'symmetry_perimeter_interaction',    # tbp_lv_symm_2axis       * tbp_lv_perimeterMM
    'comprehensive_lesion_index',        # tbp_lv_area_perim_ratio + tbp_lv_eccentricity + bp_lv_norm_color + tbp_lv_symm_2axis   / 4
    'color_variance_ratio',              # tbp_lv_color_std_mean   / tbp_lv_stdLExt
    'border_color_interaction',          # tbp_lv_norm_border      * tbp_lv_norm_color
    'border_color_interaction_2',
    'size_color_contrast_ratio',         # clin_size_long_diam_mm  / tbp_lv_deltaLBnorm
    'age_normalized_nevi_confidence',    # tbp_lv_nevi_confidence  / age_approx
    'age_normalized_nevi_confidence_2',
    'color_asymmetry_index',             # tbp_lv_symm_2axis       * tbp_lv_radial_color_std_max

    'volume_approximation_3d',           # tbp_lv_areaMM2          * sqrt(tbp_lv_x**2 + tbp_lv_y**2 + tbp_lv_z**2)
    'color_range',                       # abs(tbp_lv_L - tbp_lv_Lext) + abs(tbp_lv_A - tbp_lv_Aext) + abs(tbp_lv_B - tbp_lv_Bext)
    'shape_color_consistency',           # tbp_lv_eccentricity     * tbp_lv_color_std_mean
    'border_length_ratio',               # tbp_lv_perimeterMM      / pi * sqrt(tbp_lv_areaMM2 / pi)
    'age_size_symmetry_index',           # age_approx              * clin_size_long_diam_mm * tbp_lv_symm_2axis
    'index_age_size_symmetry',           # age_approx              * tbp_lv_areaMM2 * tbp_lv_symm_2axis
        
    'size_age_interaction2',             # sqrt(tbp_lv_minorAxisMM**2 + clin_size_long_diam_mm**2) * age_approx
    'normalized_lesion_size2',           # (clin_size_long_diam_mm * age_approx)/(clin_size_long_diam_mm + age_approx),
    'np0',                               # (sqrt((tbp_lv_areaMM2 + tbp_lv_perimeterMM + tbp_lv_minorAxisMM) / 3)) / sqrt(tbp_lv_x**2 + tbp_lv_y**2 + tbp_lv_z**2)
    'texture_homogeneity',               # 1 / tbp_lv_color_std_mean
    'shape_elongation_index',            # tbp_lv_perimeterMM / tbp_lv_minorAxisMM
    'color_balance_ratio',               # (tbp_lv_L / tbp_lv_Lext) * (tbp_lv_A / tbp_lv_Aext) * (tbp_lv_B / tbp_lv_Bext)
    'asymmetry_index',                   # tbp_lv_symm_2axis / tbp_lv_symm_2axis_angle
]


In [17]:
cat_cols = ['sex', 'anatom_site_general', 'tbp_tile_type', 'tbp_lv_location', 'tbp_lv_location_simple', 'attribution']
norm_cols = [f'{col}_patient_norm' for col in num_cols + new_num_cols]
special_cols = ['count_per_patient']
image_cols = ["target_Swin_positive_distribition_change_0.153",
              "target_Swin_past_data_argumentation_0.155",
              "target_Convnext_positive_distribition_change_0.157",
              "target_Convnext_past_data_argumentation_0.156",
              "target_Swin_efficientnet_B0_positive_distribition_change_0.147",
              "target_Swin_Best_0.158",
              "target_Swin_Modal_Fusion_0.153",
              "target_Convnext_Modal_Fusion_0.147"
             ]

feature_cols = num_cols + new_num_cols + cat_cols + norm_cols + special_cols

# 数据集修改

首先，添加了MICCAI中#完整版FInal code的缺失值补全以及新的7个特征，在代码注释中用1表示

1构造的新特征texture_homogeneity和asymmetry_index包含inf，模仿baseline的做法，添加除零保护err




<span style="color: red;">在这里，对原始代码进行修改</span>
    
<span style="color: red;">read_data中不再填补缺失值</span>
    
<span style="color: red;">添加新的函数填补缺失值，如果是test data，需要用train data的中位数填补缺失值</span>


In [18]:
def read_data(path):
    ### 此处，使用1的修改，添加填补缺失值
    df = pd.read_csv(path)
    df['sex'] = df['sex'].fillna('Unknown')
    
    df['age_approx'] = df['age_approx'].replace('NA', np.nan).astype(float)
    return (
        pl.from_pandas(df)
        .with_columns(
            lesion_size_ratio              = pl.col('tbp_lv_minorAxisMM') / pl.col('clin_size_long_diam_mm'),
            lesion_shape_index             = pl.col('tbp_lv_areaMM2') / (pl.col('tbp_lv_perimeterMM') ** 2),
            hue_contrast                   = (pl.col('tbp_lv_H') - pl.col('tbp_lv_Hext')).abs(),
            luminance_contrast             = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs(),
            lesion_color_difference        = (pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2).sqrt(),
            border_complexity              = pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_symm_2axis'),
            color_uniformity               = pl.col('tbp_lv_color_std_mean') / (pl.col('tbp_lv_radial_color_std_max') + err),
        )
        .with_columns(
            position_distance_3d           = (pl.col('tbp_lv_x') ** 2 + pl.col('tbp_lv_y') ** 2 + pl.col('tbp_lv_z') ** 2).sqrt(),
            perimeter_to_area_ratio        = pl.col('tbp_lv_perimeterMM') / pl.col('tbp_lv_areaMM2'),
            area_to_perimeter_ratio        = pl.col('tbp_lv_areaMM2') / pl.col('tbp_lv_perimeterMM'),
            lesion_visibility_score        = pl.col('tbp_lv_deltaLBnorm') + pl.col('tbp_lv_norm_color'),
            combined_anatomical_site       = pl.col('anatom_site_general') + '_' + pl.col('tbp_lv_location'),
            symmetry_border_consistency    = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border'),
            consistency_symmetry_border    = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border') / (pl.col('tbp_lv_symm_2axis') + pl.col('tbp_lv_norm_border') + err),
        )
        .with_columns(
            color_consistency              = pl.col('tbp_lv_stdL') / pl.col('tbp_lv_Lext'),
            consistency_color              = pl.col('tbp_lv_stdL') * pl.col('tbp_lv_Lext') / (pl.col('tbp_lv_stdL') + pl.col('tbp_lv_Lext')),
            size_age_interaction           = pl.col('clin_size_long_diam_mm') * pl.col('age_approx'),
            hue_color_std_interaction      = pl.col('tbp_lv_H') * pl.col('tbp_lv_color_std_mean'),
            lesion_severity_index          = (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_eccentricity')) / 3,
            shape_complexity_index         = pl.col('border_complexity') + pl.col('lesion_shape_index'),
            color_contrast_index           = pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL') + pl.col('tbp_lv_deltaLBnorm'),
        )
        .with_columns(
            log_lesion_area                = (pl.col('tbp_lv_areaMM2') + 1).log(),
            normalized_lesion_size         = pl.col('clin_size_long_diam_mm') / pl.col('age_approx'),
            mean_hue_difference            = (pl.col('tbp_lv_H') + pl.col('tbp_lv_Hext')) / 2,
            std_dev_contrast               = ((pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2) / 3).sqrt(),
            color_shape_composite_index    = (pl.col('tbp_lv_color_std_mean') + pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_symm_2axis')) / 3,
            lesion_orientation_3d          = pl.arctan2(pl.col('tbp_lv_y'), pl.col('tbp_lv_x')),
            overall_color_difference       = (pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL')) / 3,
        )
        .with_columns(
            symmetry_perimeter_interaction = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_perimeterMM'),
            comprehensive_lesion_index     = (pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_eccentricity') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_symm_2axis')) / 4,
            color_variance_ratio           = pl.col('tbp_lv_color_std_mean') / pl.col('tbp_lv_stdLExt'),
            border_color_interaction       = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color'),
            border_color_interaction_2     = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color') / (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color')),
            size_color_contrast_ratio      = pl.col('clin_size_long_diam_mm') / pl.col('tbp_lv_deltaLBnorm'),
            age_normalized_nevi_confidence = pl.col('tbp_lv_nevi_confidence') / pl.col('age_approx'),
            age_normalized_nevi_confidence_2 = (pl.col('clin_size_long_diam_mm')**2 + pl.col('age_approx')**2).sqrt(),
            color_asymmetry_index          = pl.col('tbp_lv_radial_color_std_max') * pl.col('tbp_lv_symm_2axis'),
        )
        .with_columns(
            volume_approximation_3d        = pl.col('tbp_lv_areaMM2') * (pl.col('tbp_lv_x')**2 + pl.col('tbp_lv_y')**2 + pl.col('tbp_lv_z')**2).sqrt(),
            color_range                    = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs() + (pl.col('tbp_lv_A') - pl.col('tbp_lv_Aext')).abs() + (pl.col('tbp_lv_B') - pl.col('tbp_lv_Bext')).abs(),
            shape_color_consistency        = pl.col('tbp_lv_eccentricity') * pl.col('tbp_lv_color_std_mean'),
            border_length_ratio            = pl.col('tbp_lv_perimeterMM') / (2 * np.pi * (pl.col('tbp_lv_areaMM2') / np.pi).sqrt()),
            age_size_symmetry_index        = pl.col('age_approx') * pl.col('clin_size_long_diam_mm') * pl.col('tbp_lv_symm_2axis'),
            index_age_size_symmetry        = pl.col('age_approx') * pl.col('tbp_lv_areaMM2') * pl.col('tbp_lv_symm_2axis'),
        )
        ### 此处，使用1的修改，添加7个特征
        .with_columns(
            size_age_interaction2          = ((pl.col('tbp_lv_minorAxisMM')**2 + pl.col('clin_size_long_diam_mm')**2)).sqrt() * pl.col('age_approx'),
            normalized_lesion_size2        = (pl.col('clin_size_long_diam_mm') * pl.col('age_approx'))/(pl.col('clin_size_long_diam_mm') + pl.col('age_approx')),
            np0                            = (pl.col('tbp_lv_areaMM2') + pl.col('tbp_lv_perimeterMM') + pl.col('tbp_lv_minorAxisMM')).sqrt() / (pl.col('tbp_lv_x')**2 + pl.col('tbp_lv_y')**2 + pl.col('tbp_lv_z')**2).sqrt(),
            texture_homogeneity            = 1 / (pl.col('tbp_lv_color_std_mean') + err),
            shape_elongation_index         = pl.col('tbp_lv_perimeterMM') / pl.col('tbp_lv_minorAxisMM'),
            color_balance_ratio            = (pl.col('tbp_lv_L') / pl.col('tbp_lv_Lext')) * (pl.col('tbp_lv_A') / pl.col('tbp_lv_Aext')) * (pl.col('tbp_lv_B') / pl.col('tbp_lv_Bext')),
            asymmetry_index                = pl.col('tbp_lv_symm_2axis') / (pl.col('tbp_lv_symm_2axis_angle') + err),
        )
        .with_columns(
            ((pl.col(col) - pl.col(col).mean().over('patient_id')) / (pl.col(col).std().over('patient_id') + err)).alias(f'{col}_patient_norm') for col in (num_cols + new_num_cols)
        )
        .with_columns(
            count_per_patient = pl.col('isic_id').count().over('patient_id'),
        )
        .with_columns(
            pl.col(cat_cols).cast(pl.Categorical),
        )
        .to_pandas()
        .set_index(id_col)
    )

In [19]:
def Fill_NAN(df_train, df_test):
    # 仅对浮点数类型的列进行处理
    float_cols_train = df_train.select_dtypes(include=[np.float64, np.float32]).columns
    float_cols_test = df_test.select_dtypes(include=[np.float64, np.float32]).columns
    
    # 填补 df_train 中的缺失值
    for col in float_cols_train:
        median_value = df_train[col].median()
        df_train[col] = df_train[col].fillna(median_value)
    
    # 填补 df_test 中的缺失值
    for col in float_cols_test:
        median_value = df_train[col].median()
        df_test[col] = df_test[col].fillna(median_value)
        
    return df_train, df_test
    

In [20]:
def preprocess(df_train, df_test):
    df_train, df_test = Fill_NAN(df_train, df_test)
    
    global cat_cols
    
    encoder = OneHotEncoder(sparse_output=False, dtype=np.int32, handle_unknown='ignore')
    encoder.fit(df_train[cat_cols])
    
    new_cat_cols = [f'onehot_{i}' for i in range(len(encoder.get_feature_names_out()))]

    df_train[new_cat_cols] = encoder.transform(df_train[cat_cols])
    df_train[new_cat_cols] = df_train[new_cat_cols].astype('category')

    df_test[new_cat_cols] = encoder.transform(df_test[cat_cols])
    df_test[new_cat_cols] = df_test[new_cat_cols].astype('category')  
    
    # Swin_positive_distribition_change_0.153
    df_ = pd.read_csv("/kaggle/input/miccai-isic/submission1_Swin_positive_distribition_change_0.153.csv")
    df_train = df_train.reset_index(drop=True)
    df_ = df_.reset_index(drop=True)
    df_train["target_Swin_positive_distribition_change_0.153"] = df_["target_Swin_positive_distribition_change_0.153"]
    df_ = pd.read_csv("submission1_Swin_positive_distribition_change_0.153.csv")
    df_test = df_test.reset_index(drop=True)
    df_ = df_.reset_index(drop=True)
    df_test["target_Swin_positive_distribition_change_0.153"] = df_["target_Swin_positive_distribition_change_0.153"]
        
    # Swin_past_data_argumentation_0.155
    df_ = pd.read_csv("/kaggle/input/miccai-isic/submission2_Swin_past_data_argumentation_0.155.csv")
    df_train = df_train.reset_index(drop=True)
    df_ = df_.reset_index(drop=True)
    df_train["target_Swin_past_data_argumentation_0.155"] = df_["target_Swin_past_data_argumentation_0.155"]
    df_ = pd.read_csv("submission2_Swin_past_data_argumentation_0.155.csv")
    df_test = df_test.reset_index(drop=True)
    df_ = df_.reset_index(drop=True)
    df_test["target_Swin_past_data_argumentation_0.155"] = df_["target_Swin_past_data_argumentation_0.155"]
    
    # Convnext_positive_distribition_change_0.157
    df_ = pd.read_csv("/kaggle/input/miccai-isic/submission3_Convnext_positive_distribition_change_0.157.csv")
    df_train = df_train.reset_index(drop=True)
    df_ = df_.reset_index(drop=True)
    df_train["target_Convnext_positive_distribition_change_0.157"] = df_["target_Convnext_positive_distribition_change_0.157"]
    df_ = pd.read_csv("submission3_Convnext_positive_distribition_change_0.157.csv")
    df_test = df_test.reset_index(drop=True)
    df_ = df_.reset_index(drop=True)
    df_test["target_Convnext_positive_distribition_change_0.157"] = df_["target_Convnext_positive_distribition_change_0.157"]
    
    # Convnext_past_data_argumentation_0.156
    df_ = pd.read_csv("/kaggle/input/miccai-isic/submission4_Convnext_past_data_argumentation_0.156.csv")
    df_train = df_train.reset_index(drop=True)
    df_ = df_.reset_index(drop=True)
    df_train["target_Convnext_past_data_argumentation_0.156"] = df_["target_Convnext_past_data_argumentation_0.156"]
    df_ = pd.read_csv("submission4_Convnext_past_data_argumentation_0.156.csv")
    df_test = df_test.reset_index(drop=True)
    df_ = df_.reset_index(drop=True)
    df_test["target_Convnext_past_data_argumentation_0.156"] = df_["target_Convnext_past_data_argumentation_0.156"]

    # Swin_efficientnet_B0_positive_distribition_change_0.147
    df_ = pd.read_csv("/kaggle/input/miccai-isic/submission5_Swin_efficientnet_B0_positive_distribition_change_0.147.csv")
    df_train = df_train.reset_index(drop=True)
    df_ = df_.reset_index(drop=True)
    df_train["target_Swin_efficientnet_B0_positive_distribition_change_0.147"] = df_["target_Swin_efficientnet_B0_positive_distribition_change_0.147"]
    df_ = pd.read_csv("submission5_Swin_efficientnet_B0_positive_distribition_change_0.147.csv")
    df_test = df_test.reset_index(drop=True)
    df_ = df_.reset_index(drop=True)
    df_test["target_Swin_efficientnet_B0_positive_distribition_change_0.147"] = df_["target_Swin_efficientnet_B0_positive_distribition_change_0.147"]
    
    # Swin_Best_0.158
    df_ = pd.read_csv("/kaggle/input/miccai-isic/submission6_Swin_Best_0.158.csv")
    df_train = df_train.reset_index(drop=True)
    df_ = df_.reset_index(drop=True)
    df_train["target_Swin_Best_0.158"] = df_["target_Swin_Best_0.158"]
    df_ = pd.read_csv("submission6_Swin_Best_0.158.csv")
    df_test = df_test.reset_index(drop=True)
    df_ = df_.reset_index(drop=True)
    df_test["target_Swin_Best_0.158"] = df_["target_Swin_Best_0.158"]
    
    # Swin_Modal_Fusion_0.153
    df_ = pd.read_csv("/kaggle/input/miccai-isic/submission7_Swin_Modal_Fusion_0.153.csv")
    df_train = df_train.reset_index(drop=True)
    df_ = df_.reset_index(drop=True)
    df_train["target_Swin_Modal_Fusion_0.153"] = df_["target_Swin_Modal_Fusion_0.153"]
    df_ = pd.read_csv("submission7_Swin_Modal_Fusion_0.153.csv")
    df_test = df_test.reset_index(drop=True)
    df_ = df_.reset_index(drop=True)
    df_test["target_Swin_Modal_Fusion_0.153"] = df_["target_Swin_Modal_Fusion_0.153"]
    
    # Convnext_Modal_Fusion_0.147
    df_ = pd.read_csv("/kaggle/input/miccai-isic/submission8_Convnext_Modal_Fusion_0.147.csv")
    df_train = df_train.reset_index(drop=True)
    df_ = df_.reset_index(drop=True)
    df_train["target_Convnext_Modal_Fusion_0.147"] = df_["target_Convnext_Modal_Fusion_0.147"]
    df_ = pd.read_csv("submission8_Convnext_Modal_Fusion_0.147.csv")
    df_test = df_test.reset_index(drop=True)
    df_ = df_.reset_index(drop=True)
    df_test["target_Convnext_Modal_Fusion_0.147"] = df_["target_Convnext_Modal_Fusion_0.147"]
    
    
    for col in cat_cols:
        feature_cols.remove(col)

    feature_cols.extend(new_cat_cols)
    cat_cols = new_cat_cols
    
    return df_train, df_test

In [21]:
df_train = read_data(train_path)
df_test = read_data(test_path)
df_subm = pd.read_csv(subm_path, index_col=id_col)

df_train, df_test = preprocess(df_train, df_test)

In [22]:
feature_cols_without_image_cols = copy.copy(feature_cols)
# feature_cols += image_cols

class SelectColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.columns]

In [23]:
def custom_metric(estimator, X, y_true):
    y_hat = estimator.predict_proba(X)[:, 1]
    min_tpr = 0.80
    max_fpr = abs(1 - min_tpr)
    
    v_gt = abs(y_true - 1)
    v_pred = np.array([1.0 - x for x in y_hat])
    
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    
    return partial_auc

In [24]:
# X = df_train[feature_cols]
y = df_train[target_col]
groups = df_train[group_col]
cv = StratifiedGroupKFold(5, shuffle=True, random_state = seed)

# 最佳采样比例以及特征选择

lgb_sorted_results_list = [(0.0021, 0.028), (0.0022, 0.024)]

cb_sorted_results_list = [(0.0021, 0.012), (0.0029, 0.012)]

xgb_sorted_results_list = [(0.0033, 0.012), (0.0039, 0.012)]

In [25]:
lgb_sorted_results_list = [(0.0021, 0.028), (0.0022, 0.024)]
cb_sorted_results_list = [(0.0021, 0.012), (0.0029, 0.012)]
xgb_sorted_results_list = [(0.0033, 0.012), (0.0039, 0.012)]

In [26]:
lgb_low_importance_features_dict =  {
    ('target_Swin_positive_distribition_change_0.153', (0.0021, 0.028)): [],
    ('target_Swin_positive_distribition_change_0.153', (0.0022, 0.024)): ['onehot_31','onehot_11','onehot_13','onehot_14','onehot_33','onehot_16','onehot_17','onehot_18','onehot_32','onehot_3','onehot_12','onehot_5','onehot_6','onehot_8','onehot_10','shape_complexity_index_patient_norm','onehot_0','std_dev_contrast_patient_norm','log_lesion_area','std_dev_contrast','onehot_25','onehot_42','border_length_ratio','onehot_43','onehot_36','onehot_37','onehot_38','onehot_39','onehot_21','onehot_20','tbp_lv_norm_color','onehot_28','onehot_29','onehot_22','onehot_23','onehot_24','onehot_26'],
    ('target_Swin_past_data_argumentation_0.155', (0.0021, 0.028)): [],
    ('target_Swin_past_data_argumentation_0.155', (0.0022, 0.024)): ['onehot_46', 'onehot_28', 'onehot_31', 'onehot_29', 'onehot_32', 'onehot_21', 'onehot_34', 'onehot_33', 'onehot_24', 'onehot_25', 'onehot_16', 'onehot_17', 'onehot_13', 'onehot_12', 'onehot_11', 'onehot_26', 'onehot_10', 'onehot_9', 'onehot_6', 'onehot_8', 'onehot_5', 'onehot_18', 'onehot_38', 'onehot_39', 'onehot_36', 'onehot_37', 'onehot_2', 'onehot_1', 'onehot_0', 'onehot_42', 'onehot_41'],
    ('target_Convnext_positive_distribition_change_0.157', (0.0021, 0.028)): ['onehot_42','onehot_44','onehot_32','onehot_33','onehot_34','onehot_24','onehot_21','onehot_22','onehot_25','onehot_26','onehot_23','onehot_14','onehot_12','onehot_13','std_dev_contrast_patient_norm','onehot_11','border_length_ratio','onehot_5','onehot_18','onehot_4','std_dev_contrast','onehot_15','onehot_17','onehot_16','onehot_29','onehot_28','onehot_10','log_lesion_area','onehot_8','onehot_7','onehot_6','onehot_0','onehot_35','onehot_31','onehot_37','onehot_36','onehot_39','onehot_38'],
    ('target_Convnext_positive_distribition_change_0.157', (0.0022, 0.024)): ['onehot_31','onehot_32','onehot_36','onehot_35','onehot_37','onehot_21','onehot_23','onehot_22','onehot_44','onehot_24','onehot_38','onehot_25','onehot_12','onehot_11','onehot_26','onehot_7','onehot_8','onehot_10','onehot_13','onehot_6','onehot_5','onehot_4','onehot_18','onehot_34','onehot_28','onehot_27','onehot_29','onehot_42','onehot_46','onehot_39'],
    ('target_Convnext_past_data_argumentation_0.156', (0.0021, 0.028)): [],
    ('target_Convnext_past_data_argumentation_0.156', (0.0022, 0.024)): [],
    ('target_Swin_efficientnet_B0_positive_distribition_change_0.147', (0.0021, 0.028)): ['onehot_43','onehot_46','onehot_44','onehot_33','onehot_34','onehot_21','shape_complexity_index_patient_norm','std_dev_contrast_patient_norm','std_dev_contrast','onehot_0','onehot_1','size_age_interaction2_patient_norm','onehot_22','onehot_24','onehot_25','onehot_26','onehot_27','onehot_12','onehot_13','onehot_14','onehot_16','onehot_18','onehot_8','onehot_10','onehot_11','log_lesion_area','onehot_5','onehot_28','onehot_30','onehot_31','onehot_32','tbp_lv_color_std_mean_patient_norm','onehot_36','onehot_37','onehot_38','onehot_39'],
    ('target_Swin_efficientnet_B0_positive_distribition_change_0.147', (0.0022, 0.024)): ['onehot_46','onehot_28','std_dev_contrast_patient_norm','onehot_0','border_length_ratio_patient_norm','onehot_7','onehot_8','onehot_10','onehot_11','log_lesion_area','std_dev_contrast','onehot_21','onehot_22','onehot_24','onehot_25','onehot_26','onehot_27','onehot_12','onehot_15','onehot_16','onehot_17','onehot_18','onehot_5','onehot_6','onehot_37','onehot_38','onehot_39','onehot_31','onehot_32','onehot_33','onehot_34'],
    ('target_Swin_Best_0.158', (0.0021, 0.028)): ['onehot_30','onehot_31','onehot_32','onehot_33','onehot_37','onehot_36','onehot_21','onehot_20','onehot_38','onehot_44','onehot_22','onehot_23','onehot_24','onehot_13','onehot_26','onehot_25','std_dev_contrast_patient_norm','shape_complexity_index_patient_norm','onehot_17','onehot_12','onehot_19','onehot_18','onehot_16','border_length_ratio','log_lesion_area','std_dev_contrast','onehot_2','onehot_11','onehot_8','onehot_10','onehot_7','onehot_5','onehot_43','onehot_35','onehot_39','onehot_3','onehot_29','onehot_42','onehot_28'],
    ('target_Swin_Best_0.158', (0.0022, 0.024)): [],
    ('target_Swin_Modal_Fusion_0.153', (0.0021, 0.028)): ['age_approx_patient_norm', 'onehot_8', 'onehot_9', 'onehot_10', 'onehot_22', 'consistency_color', 'hue_color_std_interaction', 'shape_complexity_index', 'log_lesion_area', 'std_dev_contrast', 'onehot_0', 'onehot_2', 'shape_complexity_index_patient_norm', 'normalized_lesion_size_patient_norm', 'area_to_perimeter_ratio_patient_norm', 'onehot_11', 'std_dev_contrast_patient_norm', 'color_variance_ratio_patient_norm', 'border_length_ratio', 'consistency_color_patient_norm', 'onehot_42', 'onehot_37', 'onehot_32', 'onehot_28', 'onehot_29', 'onehot_30', 'onehot_31', 'onehot_46', 'onehot_38', 'tbp_lv_perimeterMM_patient_norm', 'onehot_5', 'onehot_23', 'onehot_24', 'onehot_25', 'onehot_26', 'onehot_12', 'onehot_13', 'onehot_14', 'onehot_36', 'onehot_40', 'onehot_45', 'onehot_20', 'onehot_21', 'onehot_15', 'onehot_16', 'onehot_4', 'onehot_19', 'onehot_17', 'clin_size_long_diam_mm_patient_norm', 'onehot_18', 'onehot_39', 'onehot_35', 'onehot_34'],
    ('target_Swin_Modal_Fusion_0.153', (0.0022, 0.024)): [],
    ('target_Convnext_Modal_Fusion_0.147', (0.0021, 0.028)): ['onehot_46', 'onehot_41', 'onehot_26', 'onehot_28', 'onehot_19', 'onehot_23', 'onehot_22', 'onehot_21', 'onehot_25', 'luminance_contrast_patient_norm', 'onehot_42', 'onehot_24', 'onehot_10', 'onehot_13', 'onehot_12', 'onehot_11', 'onehot_4', 'shape_complexity_index_patient_norm', 'consistency_color_patient_norm', 'onehot_14', 'onehot_15', 'onehot_16', 'onehot_17', 'luminance_contrast', 'onehot_3', 'std_dev_contrast', 'std_dev_contrast_patient_norm', 'hue_color_std_interaction', 'onehot_8', 'onehot_7', 'onehot_6', 'onehot_5', 'onehot_38', 'onehot_35', 'onehot_36', 'onehot_37', 'tbp_lv_norm_color', 'onehot_33', 'onehot_18', 'log_lesion_area', 'index_age_size_symmetry_patient_norm', 'normalized_lesion_size2_patient_norm', 'border_length_ratio_patient_norm', 'onehot_0', 'onehot_32', 'onehot_31', 'onehot_29', 'shape_color_consistency', 'onehot_40', 'onehot_39', 'onehot_34'],
    ('target_Convnext_Modal_Fusion_0.147', (0.0022, 0.024)): ['onehot_41', 'onehot_20', 'onehot_21', 'onehot_28', 'onehot_46', 'onehot_0', 'onehot_2', 'onehot_22', 'onehot_23', 'onehot_24', 'onehot_25', 'onehot_26', 'onehot_12', 'onehot_8', 'onehot_10', 'onehot_11', 'onehot_13', 'onehot_14', 'onehot_4', 'onehot_5', 'onehot_6', 'onehot_7', 'onehot_15', 'onehot_16', 'onehot_17', 'onehot_18', 'onehot_19', 'onehot_43', 'onehot_42', 'onehot_32', 'onehot_33', 'onehot_34', 'onehot_35', 'onehot_29', 'onehot_31', 'onehot_36', 'onehot_37', 'onehot_38', 'onehot_39']}

In [27]:
cb_low_importance_features_dict = {
    ('target_Swin_positive_distribition_change_0.153', (0.0021, 0.012)): ['onehot_31','onehot_28','onehot_18','onehot_21','onehot_16','onehot_12','onehot_24','onehot_22','onehot_3','onehot_0'],
    ('target_Swin_positive_distribition_change_0.153', (0.0029, 0.012)): ['onehot_31','onehot_34','onehot_24','onehot_25','onehot_16','onehot_18','onehot_12','onehot_35','onehot_39','onehot_28'],
    ('target_Swin_past_data_argumentation_0.155', (0.0021, 0.012)): ['onehot_21', 'onehot_24', 'onehot_25', 'onehot_28'],
    ('target_Swin_past_data_argumentation_0.155', (0.0029, 0.012)): ['onehot_44', 'onehot_34', 'onehot_23', 'onehot_24', 'onehot_16', 'onehot_5', 'onehot_18', 'onehot_28', 'onehot_31'],
    ('target_Convnext_positive_distribition_change_0.157', (0.0021, 0.012)): [],
    ('target_Convnext_positive_distribition_change_0.157', (0.0029, 0.012)): ['onehot_35','onehot_18','onehot_25','onehot_24','onehot_28','onehot_29','onehot_4','onehot_12','onehot_31','onehot_33'],
    ('target_Convnext_past_data_argumentation_0.156', (0.0021, 0.012)): ['onehot_21','onehot_24','onehot_23','onehot_18','onehot_16','onehot_39','onehot_28'],
    ('target_Convnext_past_data_argumentation_0.156', (0.0029, 0.012)): ['onehot_22','onehot_23','onehot_24','onehot_5','onehot_18','onehot_39','onehot_0','onehot_28','onehot_31'],
    ('target_Swin_efficientnet_B0_positive_distribition_change_0.147', (0.0021, 0.012)): [],
    ('target_Swin_efficientnet_B0_positive_distribition_change_0.147', (0.0029, 0.012)): [],
    ('target_Swin_Best_0.158', (0.0021, 0.012)): ['onehot_44','onehot_39','onehot_18','onehot_19','onehot_24','onehot_28','onehot_34','onehot_31','onehot_35'],
    ('target_Swin_Best_0.158', (0.0029, 0.012)): [],
    ('target_Swin_Modal_Fusion_0.153', (0.0021, 0.012)): [],
    ('target_Swin_Modal_Fusion_0.153', (0.0029, 0.012)): ['onehot_36','onehot_21','onehot_24','onehot_12','onehot_0','onehot_16','onehot_15','onehot_13','onehot_31','onehot_19','onehot_29'],
    ('target_Convnext_Modal_Fusion_0.147', (0.0021, 0.012)): [],
    ('target_Convnext_Modal_Fusion_0.147', (0.0029, 0.012)): []}

In [28]:
xgb_low_importance_features_dict = {
    ('target_Swin_positive_distribition_change_0.153', (0.0033, 0.012)): ['onehot_42','onehot_26','onehot_18','onehot_21','onehot_22','onehot_23','onehot_24','onehot_25','onehot_10','onehot_12','onehot_11','onehot_8','onehot_30','onehot_13','onehot_15','onehot_14','onehot_16','onehot_39','onehot_5','onehot_3','onehot_31','onehot_29','onehot_28','onehot_37','onehot_36','onehot_34','onehot_33','onehot_32','onehot_38'],
    ('target_Swin_positive_distribition_change_0.153', (0.0039, 0.012)): [],
    ('target_Swin_past_data_argumentation_0.155', (0.0033, 0.012)): [],
    ('target_Swin_past_data_argumentation_0.155', (0.0039, 0.012)): ['onehot_42', 'onehot_41', 'onehot_28', 'onehot_23', 'onehot_24', 'onehot_25', 'onehot_26', 'onehot_11', 'onehot_13', 'onehot_12', 'onehot_14', 'onehot_16', 'onehot_30', 'onehot_17', 'onehot_18', 'onehot_10', 'onehot_0', 'onehot_2', 'onehot_5', 'onehot_8', 'onehot_21', 'onehot_31', 'onehot_32', 'onehot_34', 'onehot_36', 'onehot_35', 'onehot_37', 'onehot_38', 'onehot_39'],
    ('target_Convnext_positive_distribition_change_0.157', (0.0033, 0.012)): ['onehot_34','onehot_20','onehot_11','onehot_12','onehot_15','onehot_16','onehot_17','onehot_18','onehot_19','onehot_25','onehot_33','onehot_5', 'onehot_8','onehot_10','onehot_44','onehot_22','onehot_37','onehot_38','onehot_39','onehot_41','onehot_42','onehot_46','onehot_24','onehot_21','onehot_27','onehot_28','onehot_29','onehot_31','onehot_32','onehot_23'],
    ('target_Convnext_positive_distribition_change_0.157', (0.0039, 0.012)): ['onehot_21','onehot_28','onehot_25','onehot_27','onehot_24','onehot_23','onehot_22','std_dev_contrast_patient_norm','onehot_17','onehot_18','onehot_19','onehot_16','onehot_12','onehot_14','onehot_15','lesion_shape_index','onehot_44','onehot_31','onehot_32','onehot_33','onehot_35','onehot_29','onehot_36','onehot_46','texture_homogeneity','onehot_5','onehot_10','onehot_8','border_length_ratio','onehot_11','onehot_4','std_dev_contrast','onehot_39','onehot_38','onehot_42','onehot_37'],
    ('target_Convnext_past_data_argumentation_0.156', (0.0033, 0.012)): ['onehot_31','onehot_30','onehot_33','onehot_32','onehot_34','onehot_23','onehot_22','onehot_20','onehot_21','onehot_10','onehot_8','onehot_7','onehot_4','onehot_24','onehot_25','onehot_5','onehot_28','onehot_27','onehot_11','onehot_18','onehot_17','onehot_16','onehot_15','onehot_14','onehot_13','onehot_12','onehot_39','onehot_42','onehot_37','onehot_38','onehot_29','onehot_36'],
    ('target_Convnext_past_data_argumentation_0.156', (0.0039, 0.012)): [],
    ('target_Swin_efficientnet_B0_positive_distribition_change_0.147', (0.0033, 0.012)): ['onehot_43','onehot_39','onehot_26','onehot_18','onehot_27','onehot_22','onehot_21','onehot_24','onehot_25','onehot_11','onehot_16','onehot_12','onehot_15','onehot_10','onehot_38','onehot_31','onehot_32','onehot_33','onehot_34','onehot_36','onehot_35','onehot_37','onehot_28','onehot_30','onehot_1','onehot_8','onehot_5'],
    ('target_Swin_efficientnet_B0_positive_distribition_change_0.147', (0.0039, 0.012)): [],
    ('target_Swin_Best_0.158', (0.0033, 0.012)): ['onehot_12','onehot_21','onehot_19','onehot_28','onehot_29','onehot_42','luminance_contrast','onehot_8','onehot_13','std_dev_contrast_patient_norm','onehot_38','onehot_18','onehot_16','onehot_15','onehot_10','log_lesion_area','std_dev_contrast','onehot_14','onehot_39','onehot_5','onehot_31','onehot_36','onehot_37','texture_homogeneity','onehot_22','onehot_34','onehot_23','onehot_24','onehot_25','onehot_32','onehot_11'],
    ('target_Swin_Best_0.158', (0.0039, 0.012)): ['onehot_17','onehot_13','std_dev_contrast_patient_norm','onehot_22','onehot_23','onehot_24','onehot_25','onehot_39','onehot_28','log_lesion_area','onehot_14','onehot_16','onehot_27','onehot_18','onehot_19','border_length_ratio','onehot_31','onehot_35','onehot_36','onehot_37','onehot_21','texture_homogeneity','onehot_1','onehot_11','onehot_4','onehot_5','luminance_contrast','onehot_32','onehot_33','onehot_34','onehot_42','onehot_38','onehot_12','onehot_10','onehot_8','onehot_6'],
    ('target_Swin_Modal_Fusion_0.153', (0.0033, 0.012)): ['onehot_4','onehot_3','onehot_1','onehot_0','onehot_12','onehot_11','onehot_10','onehot_9','onehot_8','onehot_7','onehot_6','onehot_5','onehot_46','onehot_43','onehot_42','onehot_41','onehot_31','onehot_36','onehot_35','onehot_34','onehot_33','onehot_32','onehot_14','onehot_13','onehot_28','onehot_27','onehot_26','onehot_25','onehot_24','onehot_23','onehot_22','onehot_21','onehot_30','onehot_29','onehot_45','onehot_40','onehot_39','onehot_38','onehot_37','onehot_44','onehot_19','onehot_18','onehot_20','onehot_17','onehot_16','onehot_15'],
    ('target_Swin_Modal_Fusion_0.153', (0.0039, 0.012)): ['onehot_1','onehot_0','onehot_35','onehot_10','onehot_9','onehot_8','onehot_7','onehot_5','onehot_4','onehot_3','onehot_22','onehot_21','onehot_19','onehot_34','onehot_33','onehot_32','onehot_31','onehot_11','onehot_26','onehot_25','onehot_24','onehot_23','onehot_30','onehot_29','onehot_28','onehot_43','onehot_45','onehot_42','onehot_41','onehot_40','onehot_39','onehot_38','onehot_37','onehot_36','onehot_18','onehot_17','onehot_12','onehot_16','onehot_13','onehot_14','onehot_15'],
    ('target_Convnext_Modal_Fusion_0.147', (0.0033, 0.012)): [],
    ('target_Convnext_Modal_Fusion_0.147', (0.0039, 0.012)): ['onehot_4', 'onehot_3', 'onehot_40', 'onehot_0', 'onehot_12', 'onehot_11', 'onehot_22', 'onehot_39', 'onehot_38', 'onehot_37', 'onehot_46', 'onehot_45', 'onehot_25', 'onehot_24', 'onehot_23', 'onehot_2', 'onehot_21', 'onehot_36', 'onehot_34', 'onehot_33', 'onehot_9', 'onehot_44', 'onehot_43', 'onehot_42', 'onehot_41', 'onehot_29', 'onehot_31', 'onehot_32', 'onehot_26', 'onehot_27', 'onehot_28', 'onehot_10', 'onehot_19', 'onehot_18', 'onehot_17', 'onehot_16', 'onehot_5', 'onehot_15', 'onehot_7', 'onehot_13', 'onehot_8']}

# 根据最佳参数，训练新的模型

In [29]:
def create_data(low_importance_features, j):
    # 复制cat_cols和feature_cols
    cat_cols_copy = cat_cols[:]
    feature_cols_copy = feature_cols[:]
    feature_cols_copy += [image_cols[j]]
    feature_cols_without_image_cols_copy = feature_cols_without_image_cols[:]
    # 剔除最不显著的特征
    for feature in low_importance_features:
        if feature in cat_cols_copy:
            cat_cols_copy.remove(feature)
        feature_cols_copy.remove(feature)
        feature_cols_without_image_cols_copy.remove(feature)
    # 返回X
    X_train = df_train[feature_cols_copy]
    X_test = df_test[feature_cols_copy]
    return X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy

## LGBM

In [30]:
#lgb_model_list = []
lgb_model_prob = []

In [31]:
#target_Swin_positive_distribition_change_0.153
low_importance_features = list(lgb_low_importance_features_dict.values())[0]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 0)

Sample_ratio = (0.0021, 0.028)
Best_Params = {
    'objective':'binary',
    'verbosity':-1,
    'boosting_type':'gbdt',
    'random_state':seed,
    'n_iter': 332,
    'lambda_l1': 0.043676772187179684,
    'lambda_l2': 0.823379534442795,
    'learning_rate': 0.036104162102765944,
    'max_depth': 7,
    'num_leaves': 186,
    'colsample_bytree': 0.646431554444346,
    'colsample_bynode': 0.6689107332672238,
    'bagging_fraction': 0.9489292667696784,
    'bagging_freq': 7,
    'min_data_in_leaf': 51,
    'scale_pos_weight': 8.879573708808538
               }
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', lgb.LGBMClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#lgb_model_list.append(model)
lgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [32]:
#target_Swin_positive_distribition_change_0.153
low_importance_features = list(lgb_low_importance_features_dict.values())[1]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 0)

Sample_ratio = (0.0022, 0.024)
Best_Params = {
    'objective':'binary',
    'verbosity':-1,
    'boosting_type':'gbdt',
    'random_state':seed,
    'n_iter': 325,
    'lambda_l1': 0.3660535185654528,
    'lambda_l2': 0.24820061446870684,
    'learning_rate': 0.03051078048604547,
    'max_depth': 5,
    'num_leaves': 111,
    'colsample_bytree': 0.44751585111632136,
    'colsample_bynode': 0.865288185866792,
    'bagging_fraction': 0.8195895676487541,
    'bagging_freq': 2,
    'min_data_in_leaf': 61,
    'scale_pos_weight': 5.1922121655616245
               }
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', lgb.LGBMClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#lgb_model_list.append(model)
lgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [33]:
low_importance_features = list(lgb_low_importance_features_dict.values())[2]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 1)

Sample_ratio = (0.0021, 0.028)
Best_Params = {
    'objective':'binary',
    'verbosity':-1,
    'boosting_type':'gbdt',
    'random_state':seed,
    'n_iter': 427,
    'lambda_l1': 0.016149404835905785,
    'lambda_l2': 0.004019489615652675,
    'learning_rate': 0.01135186261720372,
    'max_depth': 8,
    'num_leaves': 88,
    'colsample_bytree': 0.6172420383484654,
    'colsample_bynode': 0.4975469443641538,
    'bagging_fraction': 0.5432811181818932,
    'bagging_freq': 5,
    'min_data_in_leaf': 80,
    'scale_pos_weight': 3.7196278930838416
               }
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', lgb.LGBMClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#lgb_model_list.append(model)
lgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [34]:
low_importance_features = list(lgb_low_importance_features_dict.values())[3]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 1)

Sample_ratio = (0.0022, 0.024)
Best_Params = {
    'objective':'binary',
    'verbosity':-1,
    'boosting_type':'gbdt',
    'random_state':seed,
    'n_iter': 581,
    'lambda_l1': 0.32580875806704834,
    'lambda_l2': 0.013149035506938224,
    'learning_rate': 0.016723299974541975,
    'max_depth': 6,
    'num_leaves': 144,
    'colsample_bytree': 0.5170794105044102,
    'colsample_bynode': 0.5829505772237593,
    'bagging_fraction': 0.8423444480731331,
    'bagging_freq': 4,
    'min_data_in_leaf': 50,
    'scale_pos_weight': 2.7821205935089797}

model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', lgb.LGBMClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#lgb_model_list.append(model)
lgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [35]:
low_importance_features = list(lgb_low_importance_features_dict.values())[4]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 2)

Sample_ratio = (0.0021, 0.028)
Best_Params = {
    'objective':'binary',
    'verbosity':-1,
    'boosting_type':'gbdt',
    'random_state':seed,
    'n_iter': 521,
    'lambda_l1': 1.064515068485073,
    'lambda_l2': 0.003790949096139092,
    'learning_rate': 0.016057299316253604,
    'max_depth': 4,
    'num_leaves': 16,
    'colsample_bytree': 0.8718436947049701,
    'colsample_bynode': 0.40124093998418886,
    'bagging_fraction': 0.4712157903908308,
    'bagging_freq': 5,
    'min_data_in_leaf': 69,
    'scale_pos_weight': 2.173489792124463
               }
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', lgb.LGBMClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#lgb_model_list.append(model)
lgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [36]:
low_importance_features = list(lgb_low_importance_features_dict.values())[5]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 2)


Sample_ratio = (0.0022, 0.024)
Best_Params = {
    'objective':'binary',
    'verbosity':-1,
    'boosting_type':'gbdt',
    'random_state':seed,
    'n_iter': 466,
    'lambda_l1': 0.1208238903032126,
    'lambda_l2': 0.7485892927179494,
    'learning_rate': 0.030982978711556232,
    'max_depth': 6,
    'num_leaves': 77,
    'colsample_bytree': 0.734494709900499,
    'colsample_bynode': 0.4465685632276665,
    'bagging_fraction': 0.8949282762536706,
    'bagging_freq': 6,
    'min_data_in_leaf': 88,
    'scale_pos_weight': 1.493789618827944}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', lgb.LGBMClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#lgb_model_list.append(model)
lgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [37]:
low_importance_features = list(lgb_low_importance_features_dict.values())[6]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 3)

Sample_ratio = (0.0021, 0.028)
Best_Params = {
    'objective':'binary',
    'verbosity':-1,
    'boosting_type':'gbdt',
    'random_state':seed,
    'n_iter': 376,
    'lambda_l1': 0.30518709247650566,
    'lambda_l2': 0.014793729530973897,
    'learning_rate': 0.015213783680258384,
    'max_depth': 8,
    'num_leaves': 119,
    'colsample_bytree': 0.7154428414067082,
    'colsample_bynode': 0.6098694347757697,
    'bagging_fraction': 0.9673249823457893,
    'bagging_freq': 6,
    'min_data_in_leaf': 134,
    'scale_pos_weight': 5.549336799439439
               }
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', lgb.LGBMClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#lgb_model_list.append(model)
lgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [38]:
low_importance_features = list(lgb_low_importance_features_dict.values())[7]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 3)

Sample_ratio = (0.0022, 0.024)
Best_Params = {
    'objective':'binary',
    'verbosity':-1,
    'boosting_type':'gbdt',
    'random_state':seed,
    'n_iter': 376,
    'lambda_l1': 0.03948386753459486,
    'lambda_l2': 0.05364454194679138,
    'learning_rate': 0.012696850948260414,
    'max_depth': 8,
    'num_leaves': 92,
    'colsample_bytree': 0.8066189321960124,
    'colsample_bynode': 0.7413198261066123,
    'bagging_fraction': 0.8506295530325217,
    'bagging_freq': 5,
    'min_data_in_leaf': 97,
    'scale_pos_weight': 7.56006234358313}

model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', lgb.LGBMClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#lgb_model_list.append(model)
lgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [39]:
low_importance_features = list(lgb_low_importance_features_dict.values())[8]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 4)

Sample_ratio = (0.0021, 0.028)
Best_Params = {
    'objective':'binary',
    'verbosity':-1,
    'boosting_type':'gbdt',
    'random_state':seed,
    'n_iter': 480,
    'lambda_l1': 0.014536295834367555,
    'lambda_l2': 0.0018000305761272889,
    'learning_rate': 0.03184650777253142,
    'max_depth': 8,
    'num_leaves': 41,
    'colsample_bytree': 0.6246524281362711,
    'colsample_bynode': 0.7833560636971553,
    'bagging_fraction': 0.7935296234661127,
    'bagging_freq': 4,
    'min_data_in_leaf': 79,
    'scale_pos_weight': 7.858762967614843}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', lgb.LGBMClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#lgb_model_list.append(model)
lgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [40]:
low_importance_features = list(lgb_low_importance_features_dict.values())[9]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 4)

Sample_ratio = (0.0022, 0.024)
Best_Params = {
    'objective':'binary',
    'verbosity':-1,
    'boosting_type':'gbdt',
    'random_state':seed,
    'n_iter': 506,
    'lambda_l1': 0.18002836840462247,
    'lambda_l2': 0.0026513125994403024,
    'learning_rate': 0.02964679006957203,
    'max_depth': 4,
    'num_leaves': 130,
    'colsample_bytree': 0.517782582106476,
    'colsample_bynode': 0.5092720331607699,
    'bagging_fraction': 0.6577039688693966,
    'bagging_freq': 7,
    'min_data_in_leaf': 33,
    'scale_pos_weight': 5.928987096071291
               }
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', lgb.LGBMClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#lgb_model_list.append(model)
lgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [41]:
low_importance_features = list(lgb_low_importance_features_dict.values())[10]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 5)

Sample_ratio = (0.0021, 0.028)
Best_Params = {
    'objective':'binary',
    'verbosity':-1,
    'boosting_type':'gbdt',
    'random_state':seed,
    'n_iter': 578,
    'lambda_l1': 0.450811602854094,
    'lambda_l2': 0.0010757099833821897,
    'learning_rate': 0.01801065323984654,
    'max_depth': 8,
    'num_leaves': 149,
    'colsample_bytree': 0.5364005936741337,
    'colsample_bynode': 0.5994010094872891,
    'bagging_fraction': 0.7857869628487191,
    'bagging_freq': 5,
    'min_data_in_leaf': 43,
    'scale_pos_weight': 9.22502336924606
               }
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', lgb.LGBMClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#lgb_model_list.append(model)
lgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [42]:
low_importance_features = list(lgb_low_importance_features_dict.values())[11]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 5)

Sample_ratio = (0.0022, 0.024)
Best_Params = {
    'objective':'binary',
    'verbosity':-1,
    'boosting_type':'gbdt',
    'random_state':seed,
    'n_iter': 464,
    'lambda_l1': 0.10952373218610092,
    'lambda_l2': 0.020389350292217306,
    'learning_rate': 0.02799159916320848,
    'max_depth': 8,
    'num_leaves': 95,
    'colsample_bytree': 0.4162593721366663,
    'colsample_bynode': 0.6298554127062207,
    'bagging_fraction': 0.7838526936122823,
    'bagging_freq': 3,
    'min_data_in_leaf': 98,
    'scale_pos_weight': 0.8106382806904615}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', lgb.LGBMClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#lgb_model_list.append(model)
lgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [43]:
low_importance_features = list(lgb_low_importance_features_dict.values())[12]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 6)

Sample_ratio = (0.0021, 0.028)
Best_Params = {
    'objective':'binary',
    'verbosity':-1,
    'boosting_type':'gbdt',
    'random_state':seed,
    'n_iter': 327,
    'lambda_l1': 0.004099907061591197,
    'lambda_l2': 0.10253466089869415,
    'learning_rate': 0.01762962604150254,
    'max_depth': 7,
    'num_leaves': 224,
    'colsample_bytree': 0.7556978908206574,
    'colsample_bynode': 0.9175398387149777,
    'bagging_fraction': 0.8731590001013972,
    'bagging_freq': 4,
    'min_data_in_leaf': 127,
    'scale_pos_weight': 1.7719081096562883
               }
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', lgb.LGBMClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#lgb_model_list.append(model)
lgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [44]:
low_importance_features = list(lgb_low_importance_features_dict.values())[13]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 6)

Sample_ratio = (0.0022, 0.024)
Best_Params = {
    'objective':'binary',
    'verbosity':-1,
    'boosting_type':'gbdt',
    'random_state':seed,
    'n_iter': 400,
    'lambda_l1': 0.017442664177207544,
    'lambda_l2': 0.0408116705548992,
    'learning_rate': 0.014809981094863176,
    'max_depth': 8,
    'num_leaves': 164,
    'colsample_bytree': 0.8282282135625061,
    'colsample_bynode': 0.6761605508670868,
    'bagging_fraction': 0.7513470924012929,
    'bagging_freq': 4,
    'min_data_in_leaf': 80,
    'scale_pos_weight': 0.8457717138588977}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', lgb.LGBMClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#lgb_model_list.append(model)
lgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [45]:
low_importance_features = list(lgb_low_importance_features_dict.values())[14]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 7)

Sample_ratio = (0.0021, 0.028)
Best_Params = {
    'objective':'binary',
    'verbosity':-1,
    'boosting_type':'gbdt',
    'random_state':seed,
    'n_iter': 254,
    'lambda_l1': 0.21759066426430257,
    'lambda_l2': 0.26965394973225204,
    'learning_rate': 0.01012548056705667,
    'max_depth': 8,
    'num_leaves': 177,
    'colsample_bytree': 0.9542697513036245,
    'colsample_bynode': 0.8579805750230943,
    'bagging_fraction': 0.9501938506710034,
    'bagging_freq': 6,
    'min_data_in_leaf': 91,
    'scale_pos_weight': 5.420866322424283}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', lgb.LGBMClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#lgb_model_list.append(model)
lgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [46]:
low_importance_features = list(lgb_low_importance_features_dict.values())[15]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 7)

Sample_ratio = (0.0022, 0.024)
Best_Params = {
    'objective':'binary',
    'verbosity':-1,
    'boosting_type':'gbdt',
    'random_state':seed,
    'n_iter': 276,
    'lambda_l1': 0.02634097069159235,
    'lambda_l2': 0.0048659010629417045,
    'learning_rate': 0.01729236520091538,
    'max_depth': 7,
    'num_leaves': 57,
    'colsample_bytree': 0.8809874894567848,
    'colsample_bynode': 0.6392669628382243,
    'bagging_fraction': 0.9887910661209663,
    'bagging_freq': 6,
    'min_data_in_leaf': 103,
    'scale_pos_weight': 2.870396462370169}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', lgb.LGBMClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#lgb_model_list.append(model)
lgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [47]:
#filter
low_importance_features = ['onehot_42','tbp_lv_area_perim_ratio','onehot_30','onehot_28','std_dev_contrast_patient_norm','onehot_22','onehot_21','border_length_ratio','onehot_1','std_dev_contrast','onehot_24','onehot_16','onehot_14','onehot_13','onehot_12','onehot_11','onehot_5','onehot_18','onehot_10','onehot_8','log_lesion_area','onehot_39','onehot_38','onehot_37','onehot_31','onehot_34','onehot_32','onehot_33']
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 0)

Sample_ratio = (0.0021, 0.028)
Best_Params =  {
    'objective':'binary',
    'verbosity':-1,
    'boosting_type':'gbdt',
    'random_state':seed,
    'n_iter': 235,
    'lambda_l1': 0.11933415193402037,
    'lambda_l2': 0.04174121230690713,
    'learning_rate': 0.017206647866223525,
    'max_depth': 7,
    'num_leaves': 180,
    'colsample_bytree': 0.9555616687494104,
    'colsample_bynode': 0.4031568322212753,
    'bagging_fraction': 0.6985723541582509,
    'bagging_freq': 5,
    'min_data_in_leaf': 103,
    'scale_pos_weight': 4.917333947812744}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('filter', SelectColumns(feature_cols_without_image_cols_copy)),
        ('classifier', lgb.LGBMClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#lgb_model_list.append(model)
lgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [48]:
#filter
low_importance_features = ['onehot_43','onehot_28','onehot_31','onehot_30','onehot_34','onehot_32','onehot_16','onehot_17','onehot_12','onehot_18','onehot_11','onehot_26','onehot_24','onehot_25','onehot_22','onehot_21','onehot_8','onehot_39','onehot_37','onehot_38','onehot_6','onehot_10','onehot_5']
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 0)

Sample_ratio = (0.0022, 0.024)
Best_Params =  {
    'objective':'binary',
    'verbosity':-1,
    'boosting_type':'gbdt',
    'random_state':seed,
    'n_iter': 427,
              'lambda_l1': 0.08027167056164403,
              'lambda_l2': 0.012356637219451504,
              'learning_rate': 0.011044873306818387,
              'max_depth': 8,
              'num_leaves': 203,
              'colsample_bytree': 0.8531494116146211,
              'colsample_bynode': 0.6380837806060741,
              'bagging_fraction': 0.5619771818677772,
              'bagging_freq': 6,
              'min_data_in_leaf': 69,
              'scale_pos_weight': 6.921323596459514}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('filter', SelectColumns(feature_cols_without_image_cols_copy)),
        ('classifier', lgb.LGBMClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#lgb_model_list.append(model)
lgb_model_prob.append(model.predict_proba(X_test)[:, 1])

## CatBoost

In [49]:
#cb_model_list = []
cb_model_prob = []

In [50]:
low_importance_features = list(cb_low_importance_features_dict.values())[0]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 0)

Sample_ratio = (0.0021, 0.012)
Best_Params = {
    'loss_function':     'Logloss',
        'verbose':           False,
        'random_state':      seed,
        'cat_features':      cat_cols_copy,
        'task_type':         'CPU',
        'bootstrap_type': 'Bernoulli',
    'iterations': 380,
         'learning_rate': 0.03073940489438025,
         'max_depth': 5,
         'l2_leaf_reg': 8.027572317746365,
         'subsample': 0.6307728195087638,
         'min_data_in_leaf': 15,
         'scale_pos_weight': 1.8621937362780616
               }

model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', cb.CatBoostClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#cb_model_list.append(model)
cb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [51]:
low_importance_features = list(cb_low_importance_features_dict.values())[1]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 0)

Sample_ratio = (0.0029, 0.012)
Best_Params = {
    'loss_function':     'Logloss',
        'verbose':           False,
        'random_state':      seed,
        'cat_features':      cat_cols_copy,
        'task_type':         'CPU',
        'bootstrap_type': 'Bernoulli',
    'iterations': 495,
         'learning_rate': 0.007321441756105811,
         'max_depth': 7,
         'l2_leaf_reg': 8.55331641799685,
         'subsample': 0.683776463421989,
         'min_data_in_leaf': 54,
         'scale_pos_weight': 1.498599939788427}

model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', cb.CatBoostClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#cb_model_list.append(model)
cb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [52]:
low_importance_features = list(cb_low_importance_features_dict.values())[2]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 1)

Sample_ratio = (0.0021, 0.012)
Best_Params = {
    'loss_function':     'Logloss',
        'verbose':           False,
        'random_state':      seed,
        'cat_features':      cat_cols_copy,
        'task_type':         'CPU',
        'bootstrap_type': 'Bernoulli',
    'iterations': 339,
         'learning_rate': 0.01560928102698264,
         'max_depth': 7,
         'l2_leaf_reg': 2.1596767124443987,
         'subsample': 0.44612130126143884,
         'min_data_in_leaf': 12,
         'scale_pos_weight': 0.8199643041850946}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', cb.CatBoostClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#cb_model_list.append(model)
cb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [53]:
low_importance_features = list(cb_low_importance_features_dict.values())[3]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 1)

Sample_ratio = (0.0029, 0.012)
Best_Params = {
    'loss_function':     'Logloss',
        'verbose':           False,
        'random_state':      seed,
        'cat_features':      cat_cols_copy,
        'task_type':         'CPU',
        'bootstrap_type': 'Bernoulli',
    'iterations': 434,
         'learning_rate': 0.025800793441033382,
         'max_depth': 6,
         'l2_leaf_reg': 6.259178346560784,
         'subsample': 0.4687942993656663,
         'min_data_in_leaf': 35,
         'scale_pos_weight': 1.212451943255052}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', cb.CatBoostClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#cb_model_list.append(model)
cb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [54]:
low_importance_features = list(cb_low_importance_features_dict.values())[4]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 2)

Sample_ratio = (0.0021, 0.012)
Best_Params = {
    'loss_function':     'Logloss',
        'verbose':           False,
        'random_state':      seed,
        'cat_features':      cat_cols_copy,
        'task_type':         'CPU',
        'bootstrap_type': 'Bernoulli',
    'iterations': 396,
         'learning_rate': 0.020668496150410477,
         'max_depth': 6,
         'l2_leaf_reg': 4.789393307151959,
         'subsample': 0.5049055303111413,
         'min_data_in_leaf': 62,
         'scale_pos_weight': 4.678660345848041}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', cb.CatBoostClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#cb_model_list.append(model)
cb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [55]:
low_importance_features = list(cb_low_importance_features_dict.values())[5]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 2)

Sample_ratio = (0.0029, 0.012)
Best_Params = {
    'loss_function':     'Logloss',
        'verbose':           False,
        'random_state':      seed,
        'cat_features':      cat_cols_copy,
        'task_type':         'CPU',
        'bootstrap_type': 'Bernoulli',
    'iterations': 449,
         'learning_rate': 0.011955446878218841,
         'max_depth': 8,
         'l2_leaf_reg': 6.12449471989363,
         'subsample': 0.45585185500680675,
         'min_data_in_leaf': 122,
         'scale_pos_weight': 2.3407097578331557}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', cb.CatBoostClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#cb_model_list.append(model)
cb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [56]:
low_importance_features = list(cb_low_importance_features_dict.values())[6]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 3)

Sample_ratio = (0.0021, 0.012)
Best_Params = {
    'loss_function':     'Logloss',
        'verbose':           False,
        'random_state':      seed,
        'cat_features':      cat_cols_copy,
        'task_type':         'CPU',
        'bootstrap_type': 'Bernoulli',
    'iterations': 384,
         'learning_rate': 0.006562441033903993,
         'max_depth': 4,
         'l2_leaf_reg': 4.789647653251467,
         'subsample': 0.9792462699161758,
         'min_data_in_leaf': 5,
         'scale_pos_weight': 7.683960148201992}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', cb.CatBoostClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#cb_model_list.append(model)
cb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [57]:
low_importance_features = list(cb_low_importance_features_dict.values())[7]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 3)

Sample_ratio = (0.0029, 0.012)
Best_Params = {
    'loss_function':     'Logloss',
        'verbose':           False,
        'random_state':      seed,
        'cat_features':      cat_cols_copy,
        'task_type':         'CPU',
        'bootstrap_type': 'Bernoulli',
    'iterations': 245,
         'learning_rate': 0.011369648041146874,
         'max_depth': 8,
         'l2_leaf_reg': 6.460799664501216,
         'subsample': 0.4216109861249126,
         'min_data_in_leaf': 23,
         'scale_pos_weight': 7.500087350884744}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', cb.CatBoostClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#cb_model_list.append(model)
cb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [58]:
low_importance_features = list(cb_low_importance_features_dict.values())[8]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 4)

Sample_ratio = (0.0021, 0.012)
Best_Params = {
    'loss_function':     'Logloss',
        'verbose':           False,
        'random_state':      seed,
        'cat_features':      cat_cols_copy,
        'task_type':         'CPU',
        'bootstrap_type': 'Bernoulli',
    'iterations': 378,
         'learning_rate': 0.054774608922256406,
         'max_depth': 4,
         'l2_leaf_reg': 9.910083329114313,
         'subsample': 0.564135760155542,
         'min_data_in_leaf': 43,
         'scale_pos_weight': 1.44822462345174}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', cb.CatBoostClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#cb_model_list.append(model)
cb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [59]:
low_importance_features = list(cb_low_importance_features_dict.values())[9]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 4)

Sample_ratio = (0.0029, 0.012)
Best_Params = {
    'loss_function':     'Logloss',
        'verbose':           False,
        'random_state':      seed,
        'cat_features':      cat_cols_copy,
        'task_type':         'CPU',
        'bootstrap_type': 'Bernoulli',
    'iterations': 498,
         'learning_rate': 0.010835322989786211,
         'max_depth': 7,
         'l2_leaf_reg': 7.178361480918862,
         'subsample': 0.6599628486164681,
         'min_data_in_leaf': 56,
         'scale_pos_weight': 0.8498121209883391}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', cb.CatBoostClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#cb_model_list.append(model)
cb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [60]:
low_importance_features = list(cb_low_importance_features_dict.values())[10]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 5)

Sample_ratio = (0.0021, 0.012)
Best_Params = {
    'loss_function':     'Logloss',
        'verbose':           False,
        'random_state':      seed,
        'cat_features':      cat_cols_copy,
        'task_type':         'CPU',
        'bootstrap_type': 'Bernoulli',
    'iterations': 338,
         'learning_rate': 0.06899031297965395,
         'max_depth': 6,
         'l2_leaf_reg': 7.882726244535647,
         'subsample': 0.9883349384719836,
         'min_data_in_leaf': 55,
         'scale_pos_weight': 1.2653440133266471}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', cb.CatBoostClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#cb_model_list.append(model)
cb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [61]:
low_importance_features = list(cb_low_importance_features_dict.values())[11]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 5)

Sample_ratio = (0.0029, 0.012)
Best_Params = {
    'loss_function':     'Logloss',
        'verbose':           False,
        'random_state':      seed,
        'cat_features':      cat_cols_copy,
        'task_type':         'CPU',
        'bootstrap_type': 'Bernoulli',
    'iterations': 437,
         'learning_rate': 0.01958048589531722,
         'max_depth': 5,
         'l2_leaf_reg': 6.10654316064216,
         'subsample': 0.7437255965635203,
         'min_data_in_leaf': 74,
         'scale_pos_weight': 2.736988039686199}

model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', cb.CatBoostClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#cb_model_list.append(model)
cb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [62]:
low_importance_features = list(cb_low_importance_features_dict.values())[12]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 6)

Sample_ratio = (0.0021, 0.012)
Best_Params = {
    'loss_function':     'Logloss',
        'verbose':           False,
        'random_state':      seed,
        'cat_features':      cat_cols_copy,
        'task_type':         'CPU',
        'bootstrap_type': 'Bernoulli',
    'iterations': 329,
         'learning_rate': 0.06532069557868636,
         'max_depth': 4,
         'l2_leaf_reg': 0.11071330186558445,
         'subsample': 0.959262360033362,
         'min_data_in_leaf': 71,
         'scale_pos_weight': 7.9186133995117265}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', cb.CatBoostClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#cb_model_list.append(model)
cb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [63]:
low_importance_features = list(cb_low_importance_features_dict.values())[13]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 6)

Sample_ratio = (0.0029, 0.012)
Best_Params = {
    'loss_function':     'Logloss',
        'verbose':           False,
        'random_state':      seed,
        'cat_features':     cat_cols_copy,
        'task_type':         'CPU',
        'bootstrap_type': 'Bernoulli',
    'iterations': 219,
         'learning_rate': 0.015852822263199038,
         'max_depth': 4,
         'l2_leaf_reg': 0.05677949592263397,
         'subsample': 0.8999080051270282,
         'min_data_in_leaf': 95,
         'scale_pos_weight': 8.197153742112064}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', cb.CatBoostClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#cb_model_list.append(model)
cb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [64]:
low_importance_features = list(cb_low_importance_features_dict.values())[14]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 7)

Sample_ratio = (0.0021, 0.012)
Best_Params = {
    'loss_function':     'Logloss',
        'verbose':           False,
        'random_state':      seed,
        'cat_features':      cat_cols_copy,
        'task_type':         'CPU',
        'bootstrap_type': 'Bernoulli',
    'iterations': 384,
         'learning_rate': 0.020907725448778405,
         'max_depth': 8,
         'l2_leaf_reg': 0.6287250049151916,
         'subsample': 0.6001967558530376,
         'min_data_in_leaf': 118,
         'scale_pos_weight': 1.3137525714583138}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', cb.CatBoostClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#cb_model_list.append(model)
cb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [65]:
low_importance_features = list(cb_low_importance_features_dict.values())[15]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 7)

Sample_ratio = (0.0029, 0.012)
Best_Params = {
    'loss_function':     'Logloss',
        'verbose':           False,
        'random_state':      seed,
        'cat_features':     cat_cols_copy,
        'task_type':         'CPU',
        'bootstrap_type': 'Bernoulli',
    'iterations': 452,
         'learning_rate': 0.17372550155232103,
         'max_depth': 8,
         'l2_leaf_reg': 0.2242945007256412,
         'subsample': 0.6836191164044608,
         'min_data_in_leaf': 67,
         'scale_pos_weight': 8.455104762694289}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', cb.CatBoostClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#cb_model_list.append(model)
cb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [66]:
#filter
low_importance_features = []
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 0)

Sample_ratio = (0.0021, 0.012)
Best_Params ={
    'loss_function':     'Logloss',
        'verbose':           False,
        'random_state':      seed,
        'cat_features':      cat_cols_copy,
        'task_type':         'CPU',
        'bootstrap_type': 'Bernoulli',
    'iterations': 446,
              'learning_rate': 0.023799238572235308,
              'max_depth': 8,
              'l2_leaf_reg': 9.966290680404905,
              'subsample': 0.9404282452980621,
              'min_data_in_leaf': 38,
              'scale_pos_weight': 3.1204678512938404}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('filter', SelectColumns(feature_cols_without_image_cols_copy)),####这个地方没有修改
        ('classifier', cb.CatBoostClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#cb_model_list.append(model)
cb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [67]:
#filter
low_importance_features = []
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 0)

Sample_ratio = (0.0029, 0.012)
Best_Params = {
    'loss_function':     'Logloss',
        'verbose':           False,
        'random_state':      seed,
        'cat_features':      cat_cols_copy,
        'task_type':         'CPU',
        'bootstrap_type': 'Bernoulli',
    'iterations': 444,
              'learning_rate': 0.03633526383208827,
              'max_depth': 5,
              'l2_leaf_reg': 6.698914827416246,
              'subsample': 0.673353439749384,
              'min_data_in_leaf': 40,
              'scale_pos_weight': 9.444395658616628}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('filter', SelectColumns(feature_cols_without_image_cols_copy)),####这个地方没有修改
        ('classifier', cb.CatBoostClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#cb_model_list.append(model)
cb_model_prob.append(model.predict_proba(X_test)[:, 1])

## XGBoost

In [68]:
#xgb_model_list = []
xgb_model_prob = []

In [69]:
low_importance_features = list(xgb_low_importance_features_dict.values())[0]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 0)

Sample_ratio = (0.0033, 0.012)
Best_Params = {
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       seed,
    #'tree_method':        'gpu_hist',
    'n_estimators': 282,
          'learning_rate': 0.029709566361265476,
          'max_depth': 5,
          'lambda': 0.01838751512612425,
          'alpha': 0.004996674508906865,
          'subsample': 0.48994203139977155,
          'colsample_bytree': 0.8662002637779233,
          'colsample_bynode': 0.9100511074452573,
          'scale_pos_weight': 0.8404630973216016}

# target_Swin_positive_distribition_change_0.153
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', xgb.XGBClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#xgb_model_list.append(model)
xgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [70]:
low_importance_features = list(xgb_low_importance_features_dict.values())[1]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 0)

Sample_ratio = (0.0039, 0.012)
Best_Params = {
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       seed,
    #'tree_method':        'gpu_hist',
    'n_estimators': 437,
         'learning_rate': 0.023992589867212085,
         'max_depth': 5,
         'lambda': 0.019081128810464268,
         'alpha': 1.0644293912777554,
         'subsample': 0.9590654916700938,
         'colsample_bytree': 0.7287905647112554,
         'colsample_bynode': 0.7694137830144945,
         'scale_pos_weight': 7.021234266943646
               }

# target_Swin_positive_distribition_change_0.153
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', xgb.XGBClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#xgb_model_list.append(model)
xgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [71]:
low_importance_features = list(xgb_low_importance_features_dict.values())[2]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 1)

Sample_ratio = (0.0033, 0.012)
Best_Params = {
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       seed,
    #'tree_method':        'gpu_hist',
    'n_estimators': 302,
         'learning_rate': 0.005914839331156235,
         'max_depth': 7,
         'lambda': 0.0010289361137294146,
         'alpha': 0.10255474038741189,
         'subsample': 0.9222352412848059,
         'colsample_bytree': 0.6922340319453344,
         'colsample_bynode': 0.4688666372227587,
         'scale_pos_weight': 3.249146792214433}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', xgb.XGBClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#xgb_model_list.append(model)
xgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [72]:
low_importance_features = list(xgb_low_importance_features_dict.values())[3]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 1)

Sample_ratio = (0.0039, 0.012)
Best_Params = {
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       seed,
    #'tree_method':        'gpu_hist',
    'n_estimators': 491,
         'learning_rate': 0.023979751116420867,
         'max_depth': 6,
         'lambda': 0.12541475844537284,
         'alpha': 0.06816253174333645,
         'subsample': 0.9822219069263931,
         'colsample_bytree': 0.5603886915813086,
         'colsample_bynode': 0.8733992423344165,
         'scale_pos_weight': 8.805743112328912}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', xgb.XGBClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#xgb_model_list.append(model)
xgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [73]:
low_importance_features = list(xgb_low_importance_features_dict.values())[4]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 2)

Sample_ratio = (0.0033, 0.012)
Best_Params = {
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       seed,
    #'tree_method':        'gpu_hist',
    'n_estimators': 430,
         'learning_rate': 0.02250254110112034,
         'max_depth': 5,
         'lambda': 0.008565984590330183,
         'alpha': 0.014076783903802601,
         'subsample': 0.7085829630934333,
         'colsample_bytree': 0.691531384242304,
         'colsample_bynode': 0.6402038169700537,
         'scale_pos_weight': 3.073516970643854}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', xgb.XGBClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#xgb_model_list.append(model)
xgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [74]:
low_importance_features = list(xgb_low_importance_features_dict.values())[5]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 2)

Sample_ratio = (0.0039, 0.012)
Best_Params = {
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       seed,
    #'tree_method':        'gpu_hist',
    'n_estimators': 392,
         'learning_rate': 0.016897025351015192,
         'max_depth': 7,
         'lambda': 0.5347166482650206,
         'alpha': 0.004797838735863612,
         'subsample': 0.7447101258898594,
         'colsample_bytree': 0.7262410992681644,
         'colsample_bynode': 0.5956413336671704,
         'scale_pos_weight': 1.072318269363552}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', xgb.XGBClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#xgb_model_list.append(model)
xgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [75]:
low_importance_features = list(xgb_low_importance_features_dict.values())[6]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 3)

Sample_ratio = (0.0033, 0.012)
Best_Params = {
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       seed,
    #'tree_method':        'gpu_hist',
    'n_estimators': 439,
         'learning_rate': 0.015092195557413982,
         'max_depth': 4,
         'lambda': 0.001228610231161041,
         'alpha': 0.20971934570099954,
         'subsample': 0.6978619789295397,
         'colsample_bytree': 0.6725964860438726,
         'colsample_bynode': 0.616173426632915,
         'scale_pos_weight': 3.903454784813998}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', xgb.XGBClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#xgb_model_list.append(model)
xgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [76]:
low_importance_features = list(xgb_low_importance_features_dict.values())[7]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 3)

Sample_ratio = (0.0039, 0.012)
Best_Params = {
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       seed,
    #'tree_method':        'gpu_hist',
    'n_estimators': 274,
         'learning_rate': 0.024287725826869132,
         'max_depth': 5,
         'lambda': 0.0028243842514040817,
         'alpha': 0.3639573351670552,
         'subsample': 0.9777000487202762,
         'colsample_bytree': 0.7635701837280123,
         'colsample_bynode': 0.8555854666441186,
         'scale_pos_weight': 5.72770439429976}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', xgb.XGBClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#xgb_model_list.append(model)
xgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [77]:
low_importance_features = list(xgb_low_importance_features_dict.values())[8]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 4)

Sample_ratio = (0.0033, 0.012)
Best_Params = {
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       seed,
    #'tree_method':        'gpu_hist',
    'n_estimators': 404,
         'learning_rate': 0.028309115530604948,
         'max_depth': 5,
         'lambda': 0.001411628928060616,
         'alpha': 0.12400263101555944,
         'subsample': 0.8768272161079016,
         'colsample_bytree': 0.7958146521200384,
         'colsample_bynode': 0.4843512271368824,
         'scale_pos_weight': 1.1993124547425305}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', xgb.XGBClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#xgb_model_list.append(model)
xgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [78]:
low_importance_features = list(xgb_low_importance_features_dict.values())[9]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 4)

Sample_ratio = (0.0039, 0.012)
Best_Params = {
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       seed,
    #'tree_method':        'gpu_hist',
    'n_estimators': 469,
         'learning_rate': 0.03573128091540397,
         'max_depth': 6,
         'lambda': 0.018686967311708793,
         'alpha': 0.00221358323378038,
         'subsample': 0.4746861116281793,
         'colsample_bytree': 0.6999374937812646,
         'colsample_bynode': 0.5868851721709285,
         'scale_pos_weight': 6.066974362993521}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', xgb.XGBClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#xgb_model_list.append(model)
xgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [79]:
low_importance_features = list(xgb_low_importance_features_dict.values())[10]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 5)

Sample_ratio = (0.0033, 0.012)
Best_Params = {
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       seed,
    #'tree_method':        'gpu_hist',
    'n_estimators': 363,
         'learning_rate': 0.0392530416899739,
         'max_depth': 5,
         'lambda': 0.08293350107877558,
         'alpha': 1.5971283538070682,
         'subsample': 0.6668399679987067,
         'colsample_bytree': 0.5226621448236854,
         'colsample_bynode': 0.6040663937731122,
         'scale_pos_weight': 5.252250622341856}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', xgb.XGBClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#xgb_model_list.append(model)
xgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [80]:
low_importance_features = list(xgb_low_importance_features_dict.values())[11]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 5)

Sample_ratio = (0.0039, 0.012)
Best_Params = {
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       seed,
    #'tree_method':        'gpu_hist',
    'n_estimators': 446,
         'learning_rate': 0.02270298021184978,
         'max_depth': 5,
         'lambda': 0.7660139457329442,
         'alpha': 0.0019381169619066005,
         'subsample': 0.9106735268682277,
         'colsample_bytree': 0.5216765651797679,
         'colsample_bynode': 0.6343928573393625,
         'scale_pos_weight': 1.567627228280918}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', xgb.XGBClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#xgb_model_list.append(model)
xgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [81]:
low_importance_features = list(xgb_low_importance_features_dict.values())[12]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 6)

Sample_ratio = (0.0033, 0.012)
Best_Params = {
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       seed,
    #'tree_method':        'gpu_hist',
    'n_estimators': 330,
         'learning_rate': 0.01092740446493452,
         'max_depth': 7,
         'lambda': 0.03596404165859604,
         'alpha': 0.06983626280281162,
         'subsample': 0.431702534905791,
         'colsample_bytree': 0.8347534046190254,
         'colsample_bynode': 0.7743163349462896,
         'scale_pos_weight': 5.447310475509671}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', xgb.XGBClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#xgb_model_list.append(model)
xgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [82]:
low_importance_features = list(xgb_low_importance_features_dict.values())[13]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 6)

Sample_ratio = (0.0039, 0.012)
Best_Params = {
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       seed,
    #'tree_method':        'gpu_hist',
    'n_estimators': 209,
         'learning_rate': 0.019987502441799726,
         'max_depth': 8,
         'lambda': 2.8484825899041226,
         'alpha': 1.695074349176365,
         'subsample': 0.8453532349836573,
         'colsample_bytree': 0.8836596431939144,
         'colsample_bynode': 0.953610824007743,
         'scale_pos_weight': 7.13739439551193}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', xgb.XGBClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#xgb_model_list.append(model)
xgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [83]:
low_importance_features = list(xgb_low_importance_features_dict.values())[14]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 7)

Sample_ratio = (0.0033, 0.012)
Best_Params = {
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       seed,
    #'tree_method':        'gpu_hist',
    'n_estimators': 214,
         'learning_rate': 0.009340228143067847,
         'max_depth': 8,
         'lambda': 0.06993023107628227,
         'alpha': 2.210873090657222,
         'subsample': 0.6229249774383616,
         'colsample_bytree': 0.8086072845669369,
         'colsample_bynode': 0.7056398252520116,
         'scale_pos_weight': 6.844149557492073}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', xgb.XGBClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#xgb_model_list.append(model)
xgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [84]:
low_importance_features = list(xgb_low_importance_features_dict.values())[15]
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 7)

Sample_ratio = (0.0039, 0.012)
Best_Params = {
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       seed,
    #'tree_method':        'gpu_hist',
    'n_estimators': 297,
         'learning_rate': 0.016222380597431516,
         'max_depth': 8,
         'lambda': 0.02939254683743037,
         'alpha': 0.029686763187940033,
         'subsample': 0.6699088516188518,
         'colsample_bytree': 0.9920044709222215,
         'colsample_bynode': 0.4105954637685134,
         'scale_pos_weight': 7.177862427154535}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('classifier', xgb.XGBClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#xgb_model_list.append(model)
xgb_model_prob.append(model.predict_proba(X_test)[:, 1])

In [85]:
low_importance_features = ['onehot_23','onehot_22','border_length_ratio','onehot_39','onehot_8','onehot_38','onehot_10','onehot_24','onehot_25','onehot_11','onehot_26','lesion_shape_index','onehot_28','onehot_31','onehot_37','onehot_36','onehot_21','log_lesion_area','std_dev_contrast_patient_norm','std_dev_contrast','onehot_3','onehot_12','onehot_34','onehot_14','onehot_16','onehot_13','onehot_17','onehot_32','onehot_18','onehot_7','onehot_19','onehot_42']
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 0)

Sample_ratio = (0.0033, 0.012)
Best_Params={
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       seed,
    #'tree_method':        'gpu_hist',
    'n_estimators': 264,
              'learning_rate': 0.02310734321962663,
              'max_depth': 6,
              'lambda': 0.0031916873429774775,
              'alpha': 0.03462659609925313,
              'subsample': 0.4336747677258443,
              'colsample_bytree': 0.9013530634929405,
              'colsample_bynode': 0.4274009281735182,
              'scale_pos_weight': 4.233005922627064}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('filter', SelectColumns(feature_cols_without_image_cols_copy)),
        ('classifier', xgb.XGBClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#xgb_model_list.append(model)
xgb_model_prob.append(model.predict_proba(X_test)[:, 1])


In [86]:
#filter
low_importance_features = []
X_train, X_test, cat_cols_copy, feature_cols_without_image_cols_copy = create_data(low_importance_features, 0)


Sample_ratio = (0.0039, 0.012)
Best_Params={
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       seed,
    #'tree_method':        'gpu_hist',
    'n_estimators': 413,
    'learning_rate': 0.025958039347901616,
    'max_depth': 5,
    'lambda': 1.2901961323361582,
    'alpha': 0.013191161570885138,
    'subsample': 0.7574116943255745,
    'colsample_bytree': 0.750960482695658,
    'colsample_bynode': 0.5036170555141674,
    'scale_pos_weight': 2.5572826397753934}
model = Pipeline([
        ('sampler_1', RandomOverSampler(sampling_strategy = Sample_ratio[0] , random_state = seed)),
        ('sampler_2', RandomUnderSampler(sampling_strategy = Sample_ratio[1], random_state = seed)),
        ('filter', SelectColumns(feature_cols_without_image_cols_copy)),
        ('classifier', xgb.XGBClassifier(**Best_Params)),
        ])
model.fit(X_train, y)
#xgb_model_list.append(model)
xgb_model_prob.append(model.predict_proba(X_test)[:, 1])

# 得到的最终30个模型的预测概率，进行加权计算，获得最终结果

In [87]:
results_list = []
prob_list = lgb_model_prob + cb_model_prob + xgb_model_prob
weights = [0, 1/40, 1/40, 1/40, 1/40, 0, 1/40, 1/40, 0, 1/40, 0, 0, 1/40, 0, 1/40, 1/40, 1/24, 1/24,
           1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 0, 0, 0, 0, 1/24, 1/24,
           1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 0, 0, 0, 0, 1/48, 1/48, 1/48, 1/48, 1/24, 1/24,
          ]
# weights = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1/6, 1/6, 
#            1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 0, 0,
#            1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 1/48, 0, 0,
#           ]
# weights = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1/6, 1/6, 
#            1/42, 1/42, 1/42, 1/42, 1/42, 1/42, 1/42, 1/42, 1/42, 1/42, 1/42, 1/42, 0,0, 0, 0, 1/42, 1/42,
#            1/42, 1/42, 1/42, 1/42, 1/42, 1/42, 1/42, 1/42, 0, 0, 0,0, 1/42, 1/42, 1/42, 1/42, 1/42, 1/42,
#           ]
# for index, prob in enumerate(prob_list, start=1):  # start=1 表示列号从1开始
#     print(f"Column {index}: {prob}")

for j in range(0, len(prob_list[0])):
    probabilities = [prob_list[j] for prob_list in prob_list]
    # average = np.average(probabilities, axis = 0)
    average = np.average(probabilities, weights = weights, axis = 0)
    results_list.append(average)
results_list
    

[0.0026142910433124564, 0.0018569460711680564, 0.002875936523715472]

<span style="color: red;">注意：此处合并概率时，默认为平均数，可以给如weight进行加权平均，可能效果更好。</span>


In [88]:
df_subm['target'] = results_list

df_subm.to_csv('submission.csv')
df_subm.head()

,target
isic_id,
ISIC_0015657,0.002614
ISIC_0015729,0.001857
ISIC_0015740,0.002876
